In [1]:
pip install statsmodels

In [2]:
import statsmodels.api as sm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib
import os
import glob
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV

## statsmodels（フィルタ後先頭30フレームの平均）

In [3]:
# 処理するフォルダのパス
folder_path = './output_moving_filtered_data'

# フォルダ内のすべてのCSVファイルを取得
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
labels = [0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,1,1,1,1,0,1,0,0,0]
i = 0
# 各ファイルに対して処理を適用
result_list = []  # 平均値を保存するリスト

for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    
    # データの読み込み
    df = pd.read_csv(file_path, low_memory=False)
    
    # `question1`から`question10`のみをフィルタリング
    filtered_df = df[df["label"].str.match(r'^question(10|[1-9])$', na=False)]
    
    # Confidenceが0.90以上の行を抽出
    filtered_df = filtered_df[filtered_df["Confidence"] >= 0.90]
    
    # `Confidence`列と`AU`で始まる列を抽出
    columns_to_extract = ["label", "Confidence"] + [col for col in filtered_df.columns if col.startswith("AU")]
    filtered_df = filtered_df[columns_to_extract]
    
    # 順序を保つためのカスタム順序のカテゴリを定義
    filtered_df["label"] = pd.Categorical(
        filtered_df["label"],
        categories=[f"question{i}" for i in range(1, 11)],
        ordered=True
    )
    
    # ラベルごとに最初の30フレームを取得
    first_30_frames_per_label = (
        filtered_df.groupby("label")
        .apply(lambda group: group.head(30))
        .reset_index(drop=True)  # グループ化インデックスをリセット
    )
    
    # 平均を計算
    mean_values_per_label = first_30_frames_per_label.groupby("label").mean(numeric_only=True)
    
    # NaNを含む行を削除
    mean_values_per_label = mean_values_per_label.dropna(axis=0)
    
    # Yを追加
    mean_values_per_label["Y"] = labels[i]
    i = i + 1
    
    # 結果をリストに保存
    result_list.append((file_name, mean_values_per_label))

# 各ファイルの結果を出力
for file_name, mean_values in result_list:
    print(f"Results for {file_name}:")
    print(mean_values)
    print()

Results for filtered_ID30_smoothed.csv:
            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.995516  0.586705  0.278403  0.464329  0.311001  0.185493   
question2     0.995715  0.642548  0.393834  0.514342  0.303340  0.173015   
question3     0.996076  0.653089  0.351344  0.467197  0.328368  0.162328   
question4     0.996124  0.611314  0.306931  0.452627  0.335895  0.182368   
question5     0.995153  0.617566  0.277745  0.575782  0.326813  0.310574   
question6     0.994046  0.619970  0.368858  0.507259  0.443115  0.232560   
question7     0.994499  0.640611  0.342359  0.465248  0.412585  0.208544   
question8     0.996331  0.572323  0.331490  0.458027  0.314261  0.291824   
question9     0.997603  0.716356  0.389402  0.485613  0.367574  0.223056   
question10    0.996228  0.739683  0.410449  0.465391  0.374358  0.198075   

                AU09      AU10      AU12      A

            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.999797  0.663449  0.300578  0.175094  0.397135  0.087894   
question2     0.998309  0.613413  0.488225  0.182503  0.466726  0.186421   
question3     0.995119  0.706113  0.342605  0.132918  0.496969  0.075051   
question4     0.993577  0.620753  0.522889  0.112564  0.599509  0.071535   
question5     0.996099  0.591205  0.511268  0.158214  0.631687  0.076867   
question6     0.998682  0.606885  0.503284  0.106865  0.606659  0.074689   
question7     0.999010  0.575101  0.364279  0.241101  0.522647  0.054403   
question8     0.996254  0.637121  0.357319  0.098621  0.524664  0.075818   
question10    0.998439  0.602762  0.296254  0.144828  0.510599  0.054888   

                AU09      AU10      AU12      AU14      AU15      AU17  \
label                                                                    
question1   0.1

            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.999036  0.795258  0.544090  0.376231  0.258706  0.081737   
question2     0.999195  0.796480  0.688940  0.183501  0.406853  0.187358   
question3     0.999353  0.848025  0.662149  0.095576  0.418222  0.230667   
question4     0.999294  0.843219  0.682084  0.173348  0.411103  0.120372   
question6     0.998167  0.800025  0.567327  0.284939  0.368199  0.142646   
question7     0.998636  0.820522  0.678695  0.237823  0.405435  0.126121   
question8     0.997464  0.743348  0.542704  0.249508  0.337536  0.101858   
question9     0.998467  0.696183  0.525646  0.246828  0.451961  0.128028   
question10    0.998583  0.789009  0.678519  0.191103  0.404184  0.237142   

                AU09      AU10      AU12      AU14      AU15      AU17  \
label                                                                    
question1   0.1

In [6]:
# 削除する列を指定
columns_to_drop = ['Confidence','label']

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # 定数列を追加
    X_train_combined = sm.add_constant(X_train_combined)
    X_test = sm.add_constant(X_test)
    
    # ロジスティック回帰モデルの作成と訓練
    model = sm.Logit(y_train_combined, X_train_combined)
    result = model.fit(disp=False)
    print(result.summary())
    
    # 予測（確率）
    y_pred_prob = result.predict(X_test)
    
    # 確率を0または1に変換
    y_pred = (y_pred_prob >= 0.5).astype(int)
    
    # 精度計算
    correct_predictions = (y_pred == y_test).sum()
    accuracy = (correct_predictions / len(y_test)) * 100  # 予測率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")  # 小数点以下2桁で%表示
    print(f"正解ラベル: {y_test.tolist()}")  # 正解ラベルをリストとして表示
    print(f"予測ラベル: {y_pred.tolist()}")  # 予測ラベルをリストとして表示

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  283
Model:                          Logit   Df Residuals:                      265
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.3522
Time:                        23:56:04   Log-Likelihood:                -122.50
converged:                       True   LL-Null:                       -189.09
Covariance Type:            nonrobust   LLR p-value:                 4.559e-20
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          9.5484      2.836      3.367      0.001       3.990      15.107
AU01         -10.8460      2.995     -3.622      0.000     -16.715      -4.977
AU02           9.1574

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  285
Model:                          Logit   Df Residuals:                      267
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.3431
Time:                        23:56:04   Log-Likelihood:                -122.11
converged:                       True   LL-Null:                       -185.88
Covariance Type:            nonrobust   LLR p-value:                 5.578e-19
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          9.4603      2.841      3.329      0.001       3.891      15.030
AU01         -10.9651      3.065     -3.578      0.000     -16.972      -4.958
AU02           9.5954      2.500      3.839      0.0

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  283
Model:                          Logit   Df Residuals:                      265
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.3573
Time:                        23:56:05   Log-Likelihood:                -121.52
converged:                       True   LL-Null:                       -189.09
Covariance Type:            nonrobust   LLR p-value:                 1.917e-20
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          9.6282      2.875      3.349      0.001       3.993      15.264
AU01         -10.6577      3.035     -3.512      0.000     -16.606      -4.710
AU02           9.1377      2.460      3.715      0.0

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  283
Model:                          Logit   Df Residuals:                      265
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.3768
Time:                        23:56:05   Log-Likelihood:                -117.84
converged:                       True   LL-Null:                       -189.09
Covariance Type:            nonrobust   LLR p-value:                 7.100e-22
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         11.4529      2.992      3.828      0.000       5.589      17.316
AU01          -9.7864      3.022     -3.239      0.001     -15.709      -3.864
AU02           7.7425      2.456      3.153      0.0

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  283
Model:                          Logit   Df Residuals:                      265
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.4134
Time:                        23:56:05   Log-Likelihood:                -110.93
converged:                       True   LL-Null:                       -189.09
Covariance Type:            nonrobust   LLR p-value:                 1.405e-24
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         12.4083      2.997      4.140      0.000       6.534      18.282
AU01         -11.8585      3.198     -3.708      0.000     -18.126      -5.591
AU02           9.2221      2.584      3.569      0.0

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  283
Model:                          Logit   Df Residuals:                      265
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.3907
Time:                        23:56:05   Log-Likelihood:                -111.99
converged:                       True   LL-Null:                       -183.81
Covariance Type:            nonrobust   LLR p-value:                 4.294e-22
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         11.0199      2.989      3.687      0.000       5.162      16.878
AU01          -9.6540      3.053     -3.162      0.002     -15.637      -3.671
AU02           8.7987      2.472      3.559      0.0

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  283
Model:                          Logit   Df Residuals:                      265
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.3476
Time:                        23:56:06   Log-Likelihood:                -123.37
converged:                       True   LL-Null:                       -189.09
Covariance Type:            nonrobust   LLR p-value:                 9.880e-20
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          9.8907      2.822      3.505      0.000       4.360      15.422
AU01         -10.4314      3.019     -3.456      0.001     -16.348      -4.515
AU02           9.0137      2.453      3.675      0.0

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# 削除する列を指定
columns_to_drop = ['Confidence', 'label', 'AU07', 'AU11', 'AU20']

# グリッドサーチ用のパラメータ設定
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],   # 正則化強度
    'penalty': ['l2'],              # L2正則化のみを使用（L1やElasticNetを使う場合はsolverを変更）
    'solver': ['lbfgs']             # lbfgsはL2正則化に対応
}

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # ロジスティック回帰モデルのグリッドサーチ
    model = LogisticRegression(max_iter=1000)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=-1)
    grid_search.fit(X_train_combined, y_train_combined)
    
    # 最適なモデルを取得
    best_model = grid_search.best_estimator_
    print(f"最適パラメータ: {grid_search.best_params_}")
    
    # テストデータで予測
    y_pred = best_model.predict(X_test)
    y_pred_prob = best_model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")
    print(f"正解ラベル: {y_test.tolist()}")
    print(f"予測ラベル: {y_pred.tolist()}")
    print(f"クラス1の確率: {y_pred_prob.tolist()}")

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")



===== テストデータ: ファイル 1/31 =====
最適パラメータ: {'C': 10, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID30_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.2601793219209849, 0.34816696592842583, 0.25657646532807293, 0.1843933445308754, 0.3817130497805213, 0.3808760812333802, 0.33428150029557285, 0.3350179633505893, 0.3245024151430544, 0.37255502037363086]

===== テストデータ: ファイル 2/31 =====
最適パラメータ: {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID31_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.03539962703744414, 0.013561030135856472, 0.0036651220201298603, 0.03750553099359853, 0.018583932549805823, 0.04852203699400595, 0.012746779090444112, 0.12412080556143056, 0.019350842947959707, 0.01467780809002285]

===== テストデータ: ファイル 3/31 =====
最適パラメータ: {'C': 10, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID32_smoothed.csv
予測率: 0.00%
正解ラベル: [1,

最適パラメータ: {'C': 10, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID50_smoothed.csv
予測率: 40.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [1, 1, 1, 1, 0, 0, 0, 1, 0, 1]
クラス1の確率: [0.9152339447076855, 0.6995269712111722, 0.6220001121125848, 0.5792584678578251, 0.3906427487960167, 0.4349046087551747, 0.45147322440331733, 0.7892028157899699, 0.39404416719500807, 0.5098569166438771]

===== テストデータ: ファイル 22/31 =====
最適パラメータ: {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID54_smoothed.csv
予測率: 11.11%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 1, 0, 0, 0]
クラス1の確率: [0.0017347160890939105, 0.0061194713555728425, 0.12218207408823224, 0.15990461285587057, 0.2712486659623888, 0.5783398866885855, 0.25379418219967376, 0.2574198967387891, 0.12981312817670307]

===== テストデータ: ファイル 23/31 =====
最適パラメータ: {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID56_smoothed.csv
予測率: 71.43%
正解ラベル: [1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 1, 1, 1, 1, 1, 0]
クラス1の確率: [0.390

## statsmodels（フィルタ後の先頭10フレームの平均）

In [5]:
# 処理するフォルダのパス
folder_path = './output_moving_filtered_data'

# フォルダ内のすべてのCSVファイルを取得
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
labels = [0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,1,1,1,1,0,1,0,0,0]
i = 0
# 各ファイルに対して処理を適用
result_list = []  # 平均値を保存するリスト

for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    
    # データの読み込み
    df = pd.read_csv(file_path, low_memory=False)
    
    # `question1`から`question10`のみをフィルタリング
    filtered_df = df[df["label"].str.match(r'^question(10|[1-9])$', na=False)]
    
    # Confidenceが0.90以上の行を抽出
    filtered_df = filtered_df[filtered_df["Confidence"] >= 0.90]
    
    # `Confidence`列と`AU`で始まる列を抽出
    columns_to_extract = ["label", "Confidence"] + [col for col in filtered_df.columns if col.startswith("AU")]
    filtered_df = filtered_df[columns_to_extract]
    
    # 順序を保つためのカスタム順序のカテゴリを定義
    filtered_df["label"] = pd.Categorical(
        filtered_df["label"],
        categories=[f"question{i}" for i in range(1, 11)],
        ordered=True
    )
    
    # ラベルごとに最初の10フレームを取得
    first_10_frames_per_label = (
        filtered_df.groupby("label")
        .apply(lambda group: group.head(10))
        .reset_index(drop=True)  # グループ化インデックスをリセット
    )
    
    # 平均を計算
    mean_values_per_label = first_10_frames_per_label.groupby("label").mean(numeric_only=True)
    
    # NaNを含む行を削除
    mean_values_per_label = mean_values_per_label.dropna(axis=0)
    
    # Yを追加
    mean_values_per_label["Y"] = labels[i]
    i = i + 1
    
    # 結果をリストに保存
    result_list.append((file_name, mean_values_per_label))

# 各ファイルの結果を出力
for file_name, mean_values in result_list:
    print(f"Results for {file_name}:")
    print(mean_values)
    print()

Results for filtered_ID30_smoothed.csv:
            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.994253  0.563476  0.291621  0.407823  0.340137  0.208585   
question2     0.995213  0.679677  0.489989  0.533290  0.270572  0.194329   
question3     0.996800  0.655168  0.335807  0.474354  0.322668  0.164400   
question4     0.996812  0.600369  0.304642  0.479000  0.312973  0.185587   
question5     0.995148  0.597816  0.281193  0.577127  0.334294  0.321020   
question6     0.995141  0.618017  0.380514  0.512854  0.444987  0.243883   
question7     0.994641  0.674597  0.359317  0.506275  0.382306  0.228034   
question8     0.997165  0.563294  0.317174  0.466524  0.315580  0.271764   
question9     0.998061  0.696176  0.394051  0.474845  0.374721  0.229086   
question10    0.996726  0.721274  0.421932  0.451504  0.436006  0.192772   

                AU09      AU10      AU12      A

            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.997788  0.809546  0.531607  0.228352  0.321553  0.208469   
question2     0.988400  0.861915  0.673838  0.267811  0.337794  0.180666   
question3     0.998567  0.863386  0.674827  0.317881  0.272294  0.435922   
question4     0.998622  0.867638  0.673032  0.391072  0.276137  0.372365   
question5     0.999158  0.810939  0.702615  0.273118  0.313012  0.373695   
question6     0.998157  0.859198  0.690260  0.338127  0.252573  0.519735   
question7     0.994185  0.724598  0.518289  0.442224  0.324276  0.172989   
question8     0.998252  0.865940  0.705246  0.257873  0.312237  0.318556   
question9     0.999532  0.833158  0.686739  0.414807  0.271479  0.558245   
question10    0.999501  0.832114  0.627855  0.527482  0.228222  0.538497   

                AU09      AU10      AU12      AU14      AU15      AU17  \
label        

In [8]:
# 削除する列を指定
columns_to_drop = ['Confidence','label']

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # 定数列を追加
    X_train_combined = sm.add_constant(X_train_combined)
    X_test = sm.add_constant(X_test)
    
    # ロジスティック回帰モデルの作成と訓練
    model = sm.Logit(y_train_combined, X_train_combined)
    result = model.fit(disp=False)
    print(result.summary())
    
    # 予測（確率）
    y_pred_prob = result.predict(X_test)
    
    # 確率を0または1に変換
    y_pred = (y_pred_prob >= 0.5).astype(int)
    
    # 精度計算
    correct_predictions = (y_pred == y_test).sum()
    accuracy = (correct_predictions / len(y_test)) * 100  # 予測率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")  # 小数点以下2桁で%表示
    print(f"正解ラベル: {y_test.tolist()}")  # 正解ラベルをリストとして表示
    print(f"予測ラベル: {y_pred.tolist()}")  # 予測ラベルをリストとして表示

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  283
Model:                          Logit   Df Residuals:                      265
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.2930
Time:                        23:56:13   Log-Likelihood:                -133.69
converged:                       True   LL-Null:                       -189.09
Covariance Type:            nonrobust   LLR p-value:                 8.526e-16
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          6.9230      2.534      2.732      0.006       1.957      11.889
AU01          -6.3781      2.411     -2.646      0.008     -11.103      -1.653
AU02           5.6511

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  285
Model:                          Logit   Df Residuals:                      267
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.2801
Time:                        23:56:13   Log-Likelihood:                -133.81
converged:                       True   LL-Null:                       -185.88
Covariance Type:            nonrobust   LLR p-value:                 1.514e-14
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          7.1555      2.522      2.837      0.005       2.212      12.099
AU01          -5.8602      2.415     -2.426      0.015     -10.594      -1.126
AU02           5.6570      1.966      2.877      0.0

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  284
Model:                          Logit   Df Residuals:                      266
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.2912
Time:                        23:56:13   Log-Likelihood:                -134.38
converged:                       True   LL-Null:                       -189.58
Covariance Type:            nonrobust   LLR p-value:                 1.015e-15
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          7.8910      2.562      3.080      0.002       2.870      12.912
AU01          -6.1815      2.443     -2.530      0.011     -10.970      -1.393
AU02           5.1206      1.952      2.623      0.0

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  283
Model:                          Logit   Df Residuals:                      265
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.2964
Time:                        23:56:13   Log-Likelihood:                -133.05
converged:                       True   LL-Null:                       -189.09
Covariance Type:            nonrobust   LLR p-value:                 4.901e-16
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          7.4178      2.545      2.915      0.004       2.430      12.406
AU01          -5.2492      2.424     -2.165      0.030     -10.001      -0.498
AU02           5.3091      1.974      2.690      0.0

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  283
Model:                          Logit   Df Residuals:                      265
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.3135
Time:                        23:56:14   Log-Likelihood:                -129.82
converged:                       True   LL-Null:                       -189.09
Covariance Type:            nonrobust   LLR p-value:                 2.918e-17
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          6.6949      2.534      2.642      0.008       1.728      11.662
AU01          -6.4822      2.424     -2.674      0.007     -11.233      -1.732
AU02           5.9462      1.979      3.004      0.0

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  286
Model:                          Logit   Df Residuals:                      268
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.2851
Time:                        23:56:14   Log-Likelihood:                -133.61
converged:                       True   LL-Null:                       -186.90
Covariance Type:            nonrobust   LLR p-value:                 5.309e-15
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          6.6230      2.560      2.587      0.010       1.605      11.641
AU01          -5.5437      2.388     -2.321      0.020     -10.225      -0.863
AU02           5.3309      1.956      2.725      0.0

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  284
Model:                          Logit   Df Residuals:                      266
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.3069
Time:                        23:56:14   Log-Likelihood:                -128.11
converged:                       True   LL-Null:                       -184.85
Covariance Type:            nonrobust   LLR p-value:                 2.675e-16
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          8.1634      2.589      3.153      0.002       3.089      13.238
AU01          -5.8375      2.453     -2.380      0.017     -10.645      -1.030
AU02           5.1926      1.976      2.628      0.0

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# 削除する列を指定
columns_to_drop = ['Confidence', 'label', 'AU07', 'AU11', 'AU20']

# グリッドサーチ用のパラメータ設定
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],   # 正則化強度
    'penalty': ['l2'],              # L2正則化のみを使用（L1やElasticNetを使う場合はsolverを変更）
    'solver': ['lbfgs']             # lbfgsはL2正則化に対応
}

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # ロジスティック回帰モデルのグリッドサーチ
    model = LogisticRegression(max_iter=1000)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=-1)
    grid_search.fit(X_train_combined, y_train_combined)
    
    # 最適なモデルを取得
    best_model = grid_search.best_estimator_
    print(f"最適パラメータ: {grid_search.best_params_}")
    
    # テストデータで予測
    y_pred = best_model.predict(X_test)
    y_pred_prob = best_model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")
    print(f"正解ラベル: {y_test.tolist()}")
    print(f"予測ラベル: {y_pred.tolist()}")
    print(f"クラス1の確率: {y_pred_prob.tolist()}")

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")



===== テストデータ: ファイル 1/31 =====
最適パラメータ: {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID30_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.30793743823097275, 0.39421812250905686, 0.20845005846223605, 0.11236806363225378, 0.38582963467967646, 0.3603299044400476, 0.32572296942989665, 0.3316154105406773, 0.3933124558637739, 0.4586160832642602]

===== テストデータ: ファイル 2/31 =====
最適パラメータ: {'C': 10, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID31_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.1590970295415259, 0.06300659925741649, 0.028129680966630936, 0.18049230380243805, 0.1273136109679226, 0.12693475438347548, 0.08475114139139224, 0.14611078294527438, 0.11538708380216019, 0.0768353078587927]

===== テストデータ: ファイル 3/31 =====
最適パラメータ: {'C': 10, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID32_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1,

最適パラメータ: {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID50_smoothed.csv
予測率: 90.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.4998975152859719, 0.49568010462112344, 0.5424888403255476, 0.4990794432257404, 0.3549962047277129, 0.43707471453336927, 0.41009119551856954, 0.4943762867654667, 0.3248934168643233, 0.36307862225859994]

===== テストデータ: ファイル 22/31 =====
最適パラメータ: {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID54_smoothed.csv
予測率: 11.11%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 1, 0, 0, 0]
クラス1の確率: [0.005260557783747103, 0.011729584615949779, 0.09073455087039849, 0.2125977717924637, 0.322930720037341, 0.6126030621868941, 0.34705268503096026, 0.20946282071403674, 0.2796293035415585]

===== テストデータ: ファイル 23/31 =====
最適パラメータ: {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID56_smoothed.csv
予測率: 71.43%
正解ラベル: [1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 1, 1, 1, 1, 1, 0]
クラス1の確率: [0.318920

## statsmodels（フィルタ後の先頭30フレームの標準偏差）

In [7]:
# 処理するフォルダのパス
folder_path = './output_moving_filtered_data'

# フォルダ内のすべてのCSVファイルを取得
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
labels = [0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,1,1,1,1,0,1,0,0,0]
i = 0
# 各ファイルに対して処理を適用
result_list = []  # 平均値を保存するリスト

for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    
    # データの読み込み
    df = pd.read_csv(file_path, low_memory=False)
    
    # `question1`から`question10`のみをフィルタリング
    filtered_df = df[df["label"].str.match(r'^question(10|[1-9])$', na=False)]
    
    # Confidenceが0.90以上の行を抽出
    filtered_df = filtered_df[filtered_df["Confidence"] >= 0.90]
    
    # `Confidence`列と`AU`で始まる列を抽出
    columns_to_extract = ["label", "Confidence"] + [col for col in filtered_df.columns if col.startswith("AU")]
    filtered_df = filtered_df[columns_to_extract]
    
    # 順序を保つためのカスタム順序のカテゴリを定義
    filtered_df["label"] = pd.Categorical(
        filtered_df["label"],
        categories=[f"question{i}" for i in range(1, 11)],
        ordered=True
    )
    
    # ラベルごとに最初の30フレームを取得
    first_30_frames_per_label = (
        filtered_df.groupby("label")
        .apply(lambda group: group.head(30))
        .reset_index(drop=True)  # グループ化インデックスをリセット
    )
    
    # 標準偏差を計算
    std_values_per_label = first_30_frames_per_label.groupby("label").std(numeric_only=True)

    # NaNを含む行を削除
    std_values_per_label = std_values_per_label.dropna(axis=0)
    
    # Yを追加
    std_values_per_label["Y"] = labels[i]
    i = i + 1
    
    # 結果をリストに保存
    result_list.append((file_name, std_values_per_label))

# 各ファイルの結果を出力
for file_name, mean_values in result_list:
    print(f"Results for {file_name}:")
    print(mean_values)
    print()

Results for filtered_ID30_smoothed.csv:
            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.001466  0.025033  0.023155  0.053183  0.023770  0.020858   
question2     0.001270  0.041890  0.075525  0.024075  0.028638  0.017322   
question3     0.001708  0.015978  0.029135  0.021304  0.016300  0.013930   
question4     0.001270  0.022869  0.011865  0.032367  0.027939  0.009538   
question5     0.001197  0.018880  0.036619  0.035231  0.017124  0.031189   
question6     0.001881  0.023875  0.014404  0.023272  0.024397  0.018919   
question7     0.002001  0.031089  0.017664  0.052384  0.042433  0.015764   
question8     0.001401  0.012390  0.038962  0.031037  0.008042  0.021269   
question9     0.001172  0.031922  0.028190  0.022848  0.031537  0.010618   
question10    0.001385  0.021430  0.019002  0.038693  0.048220  0.007847   

                AU09      AU10      AU12      A

            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.001728  0.026972  0.047023  0.045007  0.008145  0.014190   
question2     0.005791  0.033447  0.042453  0.047433  0.028352  0.020159   
question3     0.004039  0.084199  0.079634  0.072711  0.020544  0.016586   
question4     0.005118  0.126217  0.160401  0.125595  0.081470  0.014687   
question5     0.010566  0.032887  0.043560  0.066803  0.056321  0.016973   
question6     0.008610  0.029675  0.041493  0.061698  0.045242  0.039196   
question7     0.013064  0.067314  0.067849  0.065997  0.026980  0.027536   
question8     0.013665  0.058571  0.047569  0.063053  0.015814  0.012228   
question9     0.005757  0.040140  0.047712  0.041905  0.017037  0.118482   
question10    0.009308  0.032002  0.059216  0.042859  0.023932  0.028280   

                AU09      AU10      AU12      AU14      AU15      AU17  \
label        

In [10]:
# 削除する列を指定
columns_to_drop = ['Confidence','label']

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # 定数列を追加
    X_train_combined = sm.add_constant(X_train_combined)
    X_test = sm.add_constant(X_test)
    
    # ロジスティック回帰モデルの作成と訓練
    model = sm.Logit(y_train_combined, X_train_combined)
    result = model.fit(disp=False)
    print(result.summary())
    
    # 予測（確率）
    y_pred_prob = result.predict(X_test)
    
    # 確率を0または1に変換
    y_pred = (y_pred_prob >= 0.5).astype(int)
    
    # 精度計算
    correct_predictions = (y_pred == y_test).sum()
    accuracy = (correct_predictions / len(y_test)) * 100  # 予測率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")  # 小数点以下2桁で%表示
    print(f"正解ラベル: {y_test.tolist()}")  # 正解ラベルをリストとして表示
    print(f"予測ラベル: {y_pred.tolist()}")  # 予測ラベルをリストとして表示

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  283
Model:                          Logit   Df Residuals:                      265
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.1452
Time:                        23:56:22   Log-Likelihood:                -161.64
converged:                       True   LL-Null:                       -189.09
Covariance Type:            nonrobust   LLR p-value:                 7.116e-06
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.2473      0.479     -2.602      0.009      -2.187      -0.308
AU01          -2.0716      8.914     -0.232      0.816     -19.543      15.400
AU02          -7.9987

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  284
Model:                          Logit   Df Residuals:                      266
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.1380
Time:                        23:56:22   Log-Likelihood:                -163.42
converged:                       True   LL-Null:                       -189.58
Covariance Type:            nonrobust   LLR p-value:                 1.830e-05
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.3754      0.479     -2.872      0.004      -2.314      -0.437
AU01          -3.1572      8.854     -0.357      0.721     -20.511      14.197
AU02          -7.7097      8.258     -0.934      0.3

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  283
Model:                          Logit   Df Residuals:                      265
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.1577
Time:                        23:56:22   Log-Likelihood:                -159.28
converged:                       True   LL-Null:                       -189.09
Covariance Type:            nonrobust   LLR p-value:                 1.214e-06
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.6288      0.500     -3.257      0.001      -2.609      -0.649
AU01          -1.2742      9.407     -0.135      0.892     -19.711      17.162
AU02          -9.4854      8.693     -1.091      0.2

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  283
Model:                          Logit   Df Residuals:                      265
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.1269
Time:                        23:56:22   Log-Likelihood:                -165.09
converged:                       True   LL-Null:                       -189.09
Covariance Type:            nonrobust   LLR p-value:                 8.562e-05
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.3247      0.485     -2.729      0.006      -2.276      -0.373
AU01          -2.1751      8.841     -0.246      0.806     -19.504      15.154
AU02          -6.1608      8.194     -0.752      0.4

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  283
Model:                          Logit   Df Residuals:                      265
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.1422
Time:                        23:56:23   Log-Likelihood:                -162.19
converged:                       True   LL-Null:                       -189.09
Covariance Type:            nonrobust   LLR p-value:                 1.069e-05
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.5294      0.485     -3.155      0.002      -2.480      -0.579
AU01          -2.3024      9.170     -0.251      0.802     -20.276      15.671
AU02          -4.1654      8.320     -0.501      0.6

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  283
Model:                          Logit   Df Residuals:                      265
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.1476
Time:                        23:56:23   Log-Likelihood:                -156.69
converged:                       True   LL-Null:                       -183.81
Covariance Type:            nonrobust   LLR p-value:                 9.050e-06
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.4689      0.491     -2.994      0.003      -2.430      -0.507
AU01          -4.0816      9.046     -0.451      0.652     -21.812      13.648
AU02          -7.3464      8.419     -0.873      0.3

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# 削除する列を指定
columns_to_drop = ['Confidence', 'label', 'AU07', 'AU11', 'AU20']

# グリッドサーチ用のパラメータ設定
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],   # 正則化強度
    'penalty': ['l2'],              # L2正則化のみを使用（L1やElasticNetを使う場合はsolverを変更）
    'solver': ['lbfgs']             # lbfgsはL2正則化に対応
}

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # ロジスティック回帰モデルのグリッドサーチ
    model = LogisticRegression(max_iter=1000)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=-1)
    grid_search.fit(X_train_combined, y_train_combined)
    
    # 最適なモデルを取得
    best_model = grid_search.best_estimator_
    print(f"最適パラメータ: {grid_search.best_params_}")
    
    # テストデータで予測
    y_pred = best_model.predict(X_test)
    y_pred_prob = best_model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")
    print(f"正解ラベル: {y_test.tolist()}")
    print(f"予測ラベル: {y_pred.tolist()}")
    print(f"クラス1の確率: {y_pred_prob.tolist()}")

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")



===== テストデータ: ファイル 1/31 =====
最適パラメータ: {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID30_smoothed.csv
予測率: 90.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.5009190584057813, 0.30803366835643176, 0.45287756716401306, 0.39790245342177355, 0.2566546998712279, 0.2860914761129047, 0.4537312010976087, 0.36283575152671105, 0.36152503486955906, 0.4503340868788816]

===== テストデータ: ファイル 2/31 =====
最適パラメータ: {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID31_smoothed.csv
予測率: 90.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
クラス1の確率: [0.3173944354033587, 0.3311663824440597, 0.2583935195320691, 0.2889274850932202, 0.28764063937971185, 0.4932897952212026, 0.28410192747207674, 0.27086944549632885, 0.37569474652993945, 0.5128068028141158]

===== テストデータ: ファイル 3/31 =====
最適パラメータ: {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID32_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1

最適パラメータ: {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID50_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.3886315056683284, 0.3886160659925768, 0.38886754989941835, 0.3888959745451535, 0.3887353529145922, 0.38883088643615865, 0.3888513571962664, 0.38896395996268834, 0.38929624190675205, 0.38884536658018676]

===== テストデータ: ファイル 22/31 =====
最適パラメータ: {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID54_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.35562761281199373, 0.3555863877941756, 0.3558989627868372, 0.35581716754878245, 0.35559941964144925, 0.35609120018078105, 0.35567940223129224, 0.35583540182796924, 0.3555976461018114]

===== テストデータ: ファイル 23/31 =====
最適パラメータ: {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID56_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.3

## statsmodels（先頭10フレームの標準偏差）

In [21]:
# 処理するフォルダのパス
folder_path = './output_moving_filtered_data'

# フォルダ内のすべてのCSVファイルを取得
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
labels = [0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,1,1,1,1,0,1,0,0,0]
i = 0
# 各ファイルに対して処理を適用
result_list = []  # 平均値を保存するリスト

for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    
    # データの読み込み
    df = pd.read_csv(file_path, low_memory=False)
    
    # `question1`から`question10`のみをフィルタリング
    filtered_df = df[df["label"].str.match(r'^question(10|[1-9])$', na=False)]
    
    # Confidenceが0.90以上の行を抽出
    filtered_df = filtered_df[filtered_df["Confidence"] >= 0.90]
    
    # `Confidence`列と`AU`で始まる列を抽出
    columns_to_extract = ["label", "Confidence"] + [col for col in filtered_df.columns if col.startswith("AU")]
    filtered_df = filtered_df[columns_to_extract]
    
    # 順序を保つためのカスタム順序のカテゴリを定義
    filtered_df["label"] = pd.Categorical(
        filtered_df["label"],
        categories=[f"question{i}" for i in range(1, 11)],
        ordered=True
    )
    
    # ラベルごとに最初の30フレームを取得
    first_30_frames_per_label = (
        filtered_df.groupby("label")
        .apply(lambda group: group.head(10))
        .reset_index(drop=True)  # グループ化インデックスをリセット
    )
    
    # 標準偏差を計算
    std_values_per_label = first_30_frames_per_label.groupby("label").std(numeric_only=True)

    # NaNを含む行を削除
    std_values_per_label = std_values_per_label.dropna(axis=0)
    
    # Yを追加
    std_values_per_label["Y"] = labels[i]
    i = i + 1
    
    # 結果をリストに保存
    result_list.append((file_name, std_values_per_label))

# 各ファイルの結果を出力
for file_name, mean_values in result_list:
    print(f"Results for {file_name}:")
    print(mean_values)
    print()

Results for filtered_ID30_smoothed.csv:
            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.001121  0.012138  0.009317  0.023847  0.005584  0.012798   
question2     0.001382  0.023488  0.036884  0.028307  0.013806  0.007775   
question3     0.000877  0.009082  0.017248  0.013338  0.016935  0.007002   
question4     0.000845  0.016595  0.016401  0.013096  0.020036  0.009922   
question5     0.001609  0.005863  0.030660  0.044703  0.004662  0.023892   
question6     0.001885  0.002587  0.010399  0.033491  0.015164  0.015248   
question7     0.001011  0.010774  0.015984  0.033539  0.029998  0.006724   
question8     0.000721  0.007721  0.018937  0.046402  0.007213  0.004194   
question9     0.000666  0.015087  0.020192  0.007193  0.036285  0.006396   
question10    0.001114  0.006376  0.011720  0.019317  0.017930  0.006203   

                AU09      AU10      AU12      A

            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.017824  0.020348  0.016752  0.047708  0.029759  0.037944   
question2     0.023245  0.020565  0.031019  0.054746  0.035389  0.047061   
question4     0.008728  0.016276  0.025885  0.032279  0.014386  0.016699   
question5     0.012602  0.031010  0.040909  0.029881  0.008746  0.112888   
question6     0.009006  0.016382  0.012544  0.074839  0.010655  0.021697   
question7     0.007428  0.012295  0.016556  0.012696  0.009893  0.025178   
question8     0.009122  0.094269  0.113827  0.101484  0.032557  0.185679   
question9     0.005891  0.011401  0.031356  0.014165  0.020199  0.039628   
question10    0.019223  0.047033  0.061166  0.032780  0.019167  0.033760   

                AU09      AU10      AU12      AU14      AU15      AU17  \
label                                                                    
question1   0.0

In [22]:
# 削除する列を指定
columns_to_drop = ['Confidence','label']

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # 定数列を追加
    X_train_combined = sm.add_constant(X_train_combined)
    X_test = sm.add_constant(X_test)
    
    # ロジスティック回帰モデルの作成と訓練
    model = sm.Logit(y_train_combined, X_train_combined)
    result = model.fit(disp=False)
    print(result.summary())
    
    # 予測（確率）
    y_pred_prob = result.predict(X_test)
    
    # 確率を0または1に変換
    y_pred = (y_pred_prob >= 0.5).astype(int)
    
    # 精度計算
    correct_predictions = (y_pred == y_test).sum()
    accuracy = (correct_predictions / len(y_test)) * 100  # 予測率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")  # 小数点以下2桁で%表示
    print(f"正解ラベル: {y_test.tolist()}")  # 正解ラベルをリストとして表示
    print(f"予測ラベル: {y_pred.tolist()}")  # 予測ラベルをリストとして表示

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  283
Model:                          Logit   Df Residuals:                      265
Method:                           MLE   Df Model:                           17
Date:                Wed, 08 Jan 2025   Pseudo R-squ.:                  0.1095
Time:                        02:40:08   Log-Likelihood:                -168.39
converged:                       True   LL-Null:                       -189.09
Covariance Type:            nonrobust   LLR p-value:                 0.0008173
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.8638      0.464     -4.017      0.000      -2.773      -0.954
AU01           2.6245     11.941      0.220      0.826     -20.780      26.029
AU02         -16.2386

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  284
Model:                          Logit   Df Residuals:                      266
Method:                           MLE   Df Model:                           17
Date:                Wed, 08 Jan 2025   Pseudo R-squ.:                  0.1084
Time:                        02:40:09   Log-Likelihood:                -169.03
converged:                       True   LL-Null:                       -189.58
Covariance Type:            nonrobust   LLR p-value:                 0.0009036
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.9914      0.471     -4.228      0.000      -2.915      -1.068
AU01           3.3201     12.000      0.277      0.782     -20.199      26.839
AU02         -16.0489     11.712     -1.370      0.1

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  283
Model:                          Logit   Df Residuals:                      265
Method:                           MLE   Df Model:                           17
Date:                Wed, 08 Jan 2025   Pseudo R-squ.:                  0.1063
Time:                        02:40:09   Log-Likelihood:                -168.99
converged:                       True   LL-Null:                       -189.09
Covariance Type:            nonrobust   LLR p-value:                  0.001216
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.9266      0.469     -4.112      0.000      -2.845      -1.008
AU01           2.5101     11.943      0.210      0.834     -20.898      25.918
AU02         -12.7071     12.198     -1.042      0.2

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  283
Model:                          Logit   Df Residuals:                      265
Method:                           MLE   Df Model:                           17
Date:                Wed, 08 Jan 2025   Pseudo R-squ.:                  0.1001
Time:                        02:40:09   Log-Likelihood:                -170.15
converged:                       True   LL-Null:                       -189.09
Covariance Type:            nonrobust   LLR p-value:                  0.002558
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.8152      0.467     -3.883      0.000      -2.731      -0.899
AU01           6.4891     11.868      0.547      0.585     -16.771      29.750
AU02         -16.1502     11.758     -1.374      0.1

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  284
Model:                          Logit   Df Residuals:                      266
Method:                           MLE   Df Model:                           17
Date:                Wed, 08 Jan 2025   Pseudo R-squ.:                  0.1134
Time:                        02:40:10   Log-Likelihood:                -163.88
converged:                       True   LL-Null:                       -184.85
Covariance Type:            nonrobust   LLR p-value:                 0.0006878
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.9360      0.475     -4.080      0.000      -2.866      -1.006
AU01          -3.4599     12.371     -0.280      0.780     -27.706      20.787
AU02          -6.9605     12.084     -0.576      0.5

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  284
Model:                          Logit   Df Residuals:                      266
Method:                           MLE   Df Model:                           17
Date:                Wed, 08 Jan 2025   Pseudo R-squ.:                 0.08565
Time:                        02:40:10   Log-Likelihood:                -169.01
converged:                       True   LL-Null:                       -184.85
Covariance Type:            nonrobust   LLR p-value:                   0.01656
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.8909      0.468     -4.043      0.000      -2.808      -0.974
AU01           7.6418     11.915      0.641      0.521     -15.712      30.996
AU02         -17.3283     11.860     -1.461      0.1

In [23]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# 削除する列を指定
columns_to_drop = ['Confidence', 'label', 'AU07', 'AU11', 'AU20']

# グリッドサーチ用のパラメータ設定
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],   # 正則化強度
    'penalty': ['l2'],              # L2正則化のみを使用（L1やElasticNetを使う場合はsolverを変更）
    'solver': ['lbfgs']             # lbfgsはL2正則化に対応
}

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # ロジスティック回帰モデルのグリッドサーチ
    model = LogisticRegression(max_iter=1000)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=-1)
    grid_search.fit(X_train_combined, y_train_combined)
    
    # 最適なモデルを取得
    best_model = grid_search.best_estimator_
    print(f"最適パラメータ: {grid_search.best_params_}")
    
    # テストデータで予測
    y_pred = best_model.predict(X_test)
    y_pred_prob = best_model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")
    print(f"正解ラベル: {y_test.tolist()}")
    print(f"予測ラベル: {y_pred.tolist()}")
    print(f"クラス1の確率: {y_pred_prob.tolist()}")

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
最適パラメータ: {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID30_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.3886397503620818, 0.3887641432742669, 0.38859118999098535, 0.38856295873515717, 0.3886713516577634, 0.3887260539537043, 0.3886639503831105, 0.3886869918105168, 0.3887143122143924, 0.3886033766288689]

===== テストデータ: ファイル 2/31 =====
最適パラメータ: {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID31_smoothed.csv
予測率: 80.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 1, 0, 0, 0, 0, 1, 0, 0]
クラス1の確率: [0.3213867976754496, 0.2877863588024563, 0.5178893427293084, 0.3680799991635985, 0.33482199857146744, 0.2724616666360836, 0.34470539136635575, 0.5486041792269889, 0.3822276448469365, 0.34567550879000764]

===== テストデータ: ファイル 3/31 =====
最適パラメータ: {'C': 10, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID32_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1

最適パラメータ: {'C': 10, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID50_smoothed.csv
予測率: 90.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
クラス1の確率: [0.36232424036708744, 0.4100978581417467, 0.4848740245362588, 0.46412583036659005, 0.435849786466231, 0.43750460959271875, 0.47128259906173087, 0.5098087484160946, 0.45462276495249804, 0.43618743979211155]

===== テストデータ: ファイル 22/31 =====
最適パラメータ: {'C': 10, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID54_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.42923091197099017, 0.3336038132287136, 0.43837471652987126, 0.3715524830182696, 0.3563388894580133, 0.4271717520253691, 0.36830663371609185, 0.38398586082510927, 0.35651237074492814]

===== テストデータ: ファイル 23/31 =====
最適パラメータ: {'C': 10, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID56_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.388302933

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# 削除する列を指定
columns_to_drop = ['Confidence', 'label', 'AU07', 'AU11', 'AU20']

# グリッドサーチ用のパラメータ設定
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],   # 正則化強度
    'penalty': ['l2'],              # L2正則化のみを使用（L1やElasticNetを使う場合はsolverを変更）
    'solver': ['lbfgs']             # lbfgsはL2正則化に対応
}

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # ロジスティック回帰モデルのグリッドサーチ
    model = LogisticRegression(max_iter=1000)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=-1)
    grid_search.fit(X_train_combined, y_train_combined)
    
    # 最適なモデルを取得
    best_model = grid_search.best_estimator_
    print(f"最適パラメータ: {grid_search.best_params_}")
    
    # テストデータで予測
    y_pred = best_model.predict(X_test)
    y_pred_prob = best_model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")
    print(f"正解ラベル: {y_test.tolist()}")
    print(f"予測ラベル: {y_pred.tolist()}")
    print(f"クラス1の確率: {y_pred_prob.tolist()}")

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")

## statsmodels（先頭30フレームの差分）

In [3]:
# 処理するフォルダのパス
folder_path = './output_moving_filtered_data'

# フォルダ内のすべてのCSVファイルを取得
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
labels = [0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,1,1,1,1,0,1,0,0,0]
i = 0
# 各ファイルに対して処理を適用
result_list = []  # 平均値を保存するリスト

for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    
    # データの読み込み
    df = pd.read_csv(file_path, low_memory=False)
    
    # `question1`から`question10`のみをフィルタリング
    filtered_df = df[df["label"].str.match(r'^question(10|[1-9])$', na=False)]
    
    # Confidenceが0.90以上の行を抽出
    filtered_df = filtered_df[filtered_df["Confidence"] >= 0.90]
    
    # `Confidence`列と`AU`で始まる列を抽出
    columns_to_extract = ["label", "Confidence"] + [col for col in filtered_df.columns if col.startswith("AU")]
    filtered_df = filtered_df[columns_to_extract]
    
    # 順序を保つためのカスタム順序のカテゴリを定義
    filtered_df["label"] = pd.Categorical(
        filtered_df["label"],
        categories=[f"question{i}" for i in range(1, 11)],
        ordered=True
    )
    
    # ラベルごとに最初の30フレームを取得
    first_30_frames_per_label = (
        filtered_df.groupby("label")
        .apply(lambda group: group.head(30))
        .reset_index(drop=True)  # グループ化インデックスをリセット
    )
    
    # ラベルごとにフレーム間の差分を計算
    diff_values_per_label = (
        first_30_frames_per_label.groupby("label")
        .apply(lambda group: group.iloc[:, 1:].diff().dropna())
    )

    # 差分に対応するラベル列をリセット（ラベルを戻す）
    diff_values_per_label = diff_values_per_label.reset_index(level=0).dropna()
    
    # 結果をリストに保存
    result_list.append((file_name, diff_values_per_label))

    # Yを追加
    diff_values_per_label["Y"] = labels[i]
    i = i + 1
    
# 各ファイルの結果を出力
for file_name, mean_values in result_list:
    print(f"Results for {file_name}:")
    print(mean_values)
    print()

Results for filtered_ID30_smoothed.csv:
          label  Confidence      AU01      AU02      AU04      AU05      AU06  \
1     question1   -0.001724 -0.002031 -0.009685 -0.039566 -0.007552  0.002423   
2     question1    0.000788  0.004552  0.014109 -0.005977  0.017838  0.002006   
3     question1   -0.000408 -0.001043 -0.000382 -0.001480 -0.008086 -0.010739   
4     question1    0.000278 -0.014017  0.013837 -0.005446 -0.002008 -0.011532   
5     question1    0.001025 -0.010964  0.003417 -0.033513 -0.000709  0.002095   
..          ...         ...       ...       ...       ...       ...       ...   
295  question10   -0.000181 -0.010951 -0.009110 -0.025020 -0.005472  0.000085   
296  question10   -0.001860  0.000080 -0.019191  0.029150 -0.005756  0.004851   
297  question10    0.002063  0.002683 -0.009171  0.038275  0.014208 -0.010177   
298  question10   -0.003481  0.001530 -0.003313  0.003233  0.022098 -0.007301   
299  question10   -0.001027  0.009657  0.012584 -0.008357  0.011535 -

          label  Confidence      AU01      AU02      AU04      AU05      AU06  \
1     question1   -0.001328 -0.015538 -0.004013 -0.023436  0.007517  0.007463   
2     question1   -0.001988 -0.010663 -0.017819 -0.000699  0.001056  0.004963   
3     question1    0.004043 -0.030511 -0.013553  0.005301  0.008921 -0.003619   
4     question1   -0.003457 -0.021634 -0.019622 -0.006166 -0.004070 -0.005874   
5     question1   -0.010945 -0.036270 -0.018347  0.014513  0.004984 -0.006997   
..          ...         ...       ...       ...       ...       ...       ...   
295  question10   -0.000309 -0.040501 -0.027605 -0.020866  0.015581  0.018700   
296  question10   -0.000403  0.003437  0.001203  0.001467  0.011204  0.000355   
297  question10    0.000896 -0.001669  0.004751 -0.011851  0.000681  0.006118   
298  question10   -0.000413 -0.005185 -0.015997 -0.006522 -0.000740  0.011851   
299  question10   -0.001021  0.000214 -0.011155 -0.003853  0.006364  0.004650   

         AU09      AU10    

In [12]:
# 削除する列を指定
columns_to_drop = ['Confidence','label']

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # 定数列を追加
    X_train_combined = sm.add_constant(X_train_combined)
    X_test = sm.add_constant(X_test)
    
    # ロジスティック回帰モデルの作成と訓練
    model = sm.Logit(y_train_combined, X_train_combined)
    result = model.fit(disp=False)
    print(result.summary())
    
    # 予測（確率）
    y_pred_prob = result.predict(X_test)
    
    # 確率を0または1に変換
    y_pred = (y_pred_prob >= 0.5).astype(int)
    
    # 精度計算
    correct_predictions = (y_pred == y_test).sum()
    accuracy = (correct_predictions / len(y_test)) * 100  # 予測率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")  # 小数点以下2桁で%表示
    print(f"正解ラベル: {y_test.tolist()}")  # 正解ラベルをリストとして表示
    print(f"予測ラベル: {y_pred.tolist()}")  # 予測ラベルをリストとして表示

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                 8156
Model:                          Logit   Df Residuals:                     8138
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                0.001745
Time:                        23:56:30   Log-Likelihood:                -5448.9
converged:                       True   LL-Null:                       -5458.4
Covariance Type:            nonrobust   LLR p-value:                    0.3257
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.4383      0.023    -19.231      0.000      -0.483      -0.394
AU01          -2.0430      2.231     -0.916      0.360      -6.416       2.330
AU02           2.4534

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                 8185
Model:                          Logit   Df Residuals:                     8167
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                0.001753
Time:                        23:56:30   Log-Likelihood:                -5328.7
converged:                       True   LL-Null:                       -5338.0
Covariance Type:            nonrobust   LLR p-value:                    0.3451
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.5812      0.023    -25.098      0.000      -0.627      -0.536
AU01          -2.1170      2.263     -0.935      0.350      -6.553       2.319
AU02           2.8547      1.870      1.527      0.1

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                 8156
Model:                          Logit   Df Residuals:                     8138
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                0.001602
Time:                        23:56:31   Log-Likelihood:                -5449.6
converged:                       True   LL-Null:                       -5458.4
Covariance Type:            nonrobust   LLR p-value:                    0.4218
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.4390      0.023    -19.267      0.000      -0.484      -0.394
AU01          -1.8449      2.222     -0.830      0.406      -6.200       2.510
AU02           1.9267      1.850      1.041      0.2

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                 8156
Model:                          Logit   Df Residuals:                     8138
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                0.001459
Time:                        23:56:31   Log-Likelihood:                -5450.4
converged:                       True   LL-Null:                       -5458.4
Covariance Type:            nonrobust   LLR p-value:                    0.5291
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.4392      0.023    -19.279      0.000      -0.484      -0.395
AU01          -1.9023      2.222     -0.856      0.392      -6.257       2.453
AU02           2.6576      1.847      1.439      0.1

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                 8156
Model:                          Logit   Df Residuals:                     8138
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                0.001503
Time:                        23:56:31   Log-Likelihood:                -5300.1
converged:                       True   LL-Null:                       -5308.1
Covariance Type:            nonrobust   LLR p-value:                    0.5272
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.5911      0.023    -25.446      0.000      -0.637      -0.546
AU01          -2.3297      2.250     -1.035      0.301      -6.740       2.081
AU02           2.4882      1.866      1.333      0.1

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                 8167
Model:                          Logit   Df Residuals:                     8149
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                0.002092
Time:                        23:56:31   Log-Likelihood:                -5452.4
converged:                       True   LL-Null:                       -5463.8
Covariance Type:            nonrobust   LLR p-value:                    0.1539
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.4400      0.023    -19.307      0.000      -0.485      -0.395
AU01          -2.4325      2.265     -1.074      0.283      -6.873       2.008
AU02           3.3794      1.864      1.813      0.0

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                 8185
Model:                          Logit   Df Residuals:                     8167
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                0.001877
Time:                        23:56:32   Log-Likelihood:                -5328.0
converged:                       True   LL-Null:                       -5338.0
Covariance Type:            nonrobust   LLR p-value:                    0.2725
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.5810      0.023    -25.087      0.000      -0.626      -0.536
AU01          -1.5018      2.309     -0.650      0.515      -6.027       3.023
AU02           2.9145      1.893      1.539      0.1

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                 8185
Model:                          Logit   Df Residuals:                     8167
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                0.001764
Time:                        23:56:32   Log-Likelihood:                -5328.6
converged:                       True   LL-Null:                       -5338.0
Covariance Type:            nonrobust   LLR p-value:                    0.3379
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.5813      0.023    -25.101      0.000      -0.627      -0.536
AU01          -2.5649      2.256     -1.137      0.256      -6.987       1.857
AU02           3.0890      1.865      1.656      0.0

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                 8156
Model:                          Logit   Df Residuals:                     8138
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                0.001835
Time:                        23:56:32   Log-Likelihood:                -5448.4
converged:                       True   LL-Null:                       -5458.4
Covariance Type:            nonrobust   LLR p-value:                    0.2726
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.4385      0.023    -19.240      0.000      -0.483      -0.394
AU01          -2.2007      2.226     -0.988      0.323      -6.564       2.163
AU02           3.2654      1.851      1.764      0.0

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# 削除する列を指定
columns_to_drop = ['Confidence', 'label', 'AU07', 'AU11', 'AU20']

# グリッドサーチ用のパラメータ設定
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],   # 正則化強度
    'penalty': ['l2'],              # L2正則化のみを使用（L1やElasticNetを使う場合はsolverを変更）
    'solver': ['lbfgs']             # lbfgsはL2正則化に対応
}

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # ロジスティック回帰モデルのグリッドサーチ
    model = LogisticRegression(max_iter=1000)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=-1)
    grid_search.fit(X_train_combined, y_train_combined)
    
    # 最適なモデルを取得
    best_model = grid_search.best_estimator_
    print(f"最適パラメータ: {grid_search.best_params_}")
    
    # テストデータで予測
    y_pred = best_model.predict(X_test)
    y_pred_prob = best_model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")
    print(f"正解ラベル: {y_test.tolist()}")
    print(f"予測ラベル: {y_pred.tolist()}")
    print(f"クラス1の確率: {y_pred_prob.tolist()}")

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
最適パラメータ: {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID30_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

最適パラメータ: {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID32_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

最適パラメータ: {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID34_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

最適パラメータ: {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID36_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0

最適パラメータ: {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID38_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

...

最適パラメータ: {'C': 10, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID56_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

最適パラメータ: {'C': 10, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID63_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

最適パラメータ: {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID69_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0

最適パラメータ: {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID71_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0

最適パラメータ: {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID73_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0

## statsmodels（フィルタ後の先頭30フレームの差分の標準偏差）

In [9]:
# 処理するフォルダのパス
folder_path = './output_moving_filtered_data'

# フォルダ内のすべてのCSVファイルを取得
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
labels = [0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,1,1,1,1,0,1,0,0,0]
i = 0
# 各ファイルに対して処理を適用
result_list = []  # 平均値を保存するリスト

for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    
    # データの読み込み
    df = pd.read_csv(file_path, low_memory=False)
    
    # `question1`から`question10`のみをフィルタリング
    filtered_df = df[df["label"].str.match(r'^question(10|[1-9])$', na=False)]
    
    # Confidenceが0.90以上の行を抽出
    filtered_df = filtered_df[filtered_df["Confidence"] >= 0.90]
    
    # `Confidence`列と`AU`で始まる列を抽出
    columns_to_extract = ["label", "Confidence"] + [col for col in filtered_df.columns if col.startswith("AU")]
    filtered_df = filtered_df[columns_to_extract]
    
    # 順序を保つためのカスタム順序のカテゴリを定義
    filtered_df["label"] = pd.Categorical(
        filtered_df["label"],
        categories=[f"question{i}" for i in range(1, 11)],
        ordered=True
    )
    
    # ラベルごとに最初の30フレームを取得
    first_30_frames_per_label = (
        filtered_df.groupby("label")
        .apply(lambda group: group.head(30))
        .reset_index(drop=True)  # グループ化インデックスをリセット
    )
    
    # ラベルごとにフレーム間の差分を計算
    diff_values_per_label = (
        first_30_frames_per_label.groupby("label")
        .apply(lambda group: group.iloc[:, 1:].diff().dropna())
    )

    # 差分に対応するラベル列をリセット（ラベルを戻す）
    diff_values_per_label = diff_values_per_label.reset_index(level=0).dropna()
    
    # 差分の標準偏差を計算
    std_values_per_label = (
        diff_values_per_label.groupby("label").std(numeric_only=True)
    )
    
    # 結果をリストに保存
    result_list.append((file_name, std_values_per_label))

    # Yを追加
    std_values_per_label["Y"] = labels[i]
    i = i + 1
    
# 各ファイルの結果を出力
for file_name, mean_values in result_list:
    print(f"Results for {file_name}:")
    print(mean_values)
    print()

Results for filtered_ID30_smoothed.csv:
            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.001357  0.011945  0.011015  0.017625  0.008104  0.005981   
question2     0.001140  0.009808  0.013247  0.013333  0.009484  0.007334   
question3     0.001540  0.011213  0.013871  0.017764  0.010590  0.004507   
question4     0.001450  0.011179  0.011102  0.015170  0.010495  0.005445   
question5     0.001605  0.009229  0.013612  0.023189  0.007825  0.013743   
question6     0.001716  0.007972  0.011852  0.015273  0.010305  0.008880   
question7     0.002743  0.010369  0.011425  0.022203  0.015390  0.004805   
question8     0.001929  0.010254  0.017442  0.022008  0.004848  0.010167   
question9     0.001081  0.011445  0.013607  0.015146  0.010546  0.005656   
question10    0.001231  0.008036  0.010590  0.025266  0.011607  0.004519   

                AU09      AU10      AU12      A

            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.002534  0.007714  0.013640  0.014009  0.005785  0.008371   
question2     0.002673  0.010259  0.015221  0.016400  0.001820  0.007500   
question3     0.000119  0.005886  0.013060  0.020664  0.006908  0.002138   
question4     0.000109  0.004117  0.011837  0.019358  0.003635  0.001620   
question5     0.000233  0.003214  0.002974  0.011461  0.003384  0.002414   
question6     0.000044  0.005635  0.004931  0.005721  0.005186  0.002042   
question7     0.000053  0.004681  0.010799  0.014118  0.004391  0.000938   
question8     0.000044  0.003481  0.006704  0.008891  0.003564  0.000557   
question9     0.000058  0.013601  0.015158  0.014851  0.006297  0.001727   
question10    0.000063  0.009855  0.005160  0.017684  0.009508  0.000882   

                AU09      AU10      AU12      AU14      AU15      AU17  \
label        

            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.000744  0.017617  0.020472  0.027486  0.007161  0.010127   
question2     0.003587  0.023716  0.019278  0.030726  0.007407  0.016811   
question3     0.012542  0.019989  0.024368  0.035646  0.011606  0.014976   
question4     0.000583  0.023510  0.027447  0.028648  0.007347  0.001856   
question5     0.000688  0.012980  0.023866  0.031798  0.012646  0.006775   
question6     0.000931  0.023710  0.028140  0.028236  0.006110  0.007802   
question7     0.000944  0.020556  0.022295  0.035590  0.009733  0.006283   
question8     0.000663  0.033250  0.032377  0.045644  0.011760  0.004041   
question9     0.000536  0.018951  0.026978  0.037577  0.007719  0.007755   
question10    0.000849  0.025404  0.025527  0.038763  0.005785  0.001354   

                AU09      AU10      AU12      AU14      AU15      AU17  \
label        

In [14]:
# 削除する列を指定
columns_to_drop = ['Confidence','label']

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # 定数列を追加
    X_train_combined = sm.add_constant(X_train_combined)
    X_test = sm.add_constant(X_test)
    
    # ロジスティック回帰モデルの作成と訓練
    model = sm.Logit(y_train_combined, X_train_combined)
    result = model.fit(disp=False)
    print(result.summary())
    
    # 予測（確率）
    y_pred_prob = result.predict(X_test)
    
    # 確率を0または1に変換
    y_pred = (y_pred_prob >= 0.5).astype(int)
    
    # 精度計算
    correct_predictions = (y_pred == y_test).sum()
    accuracy = (correct_predictions / len(y_test)) * 100  # 予測率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")  # 小数点以下2桁で%表示
    print(f"正解ラベル: {y_test.tolist()}")  # 正解ラベルをリストとして表示
    print(f"予測ラベル: {y_pred.tolist()}")  # 予測ラベルをリストとして表示

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  283
Model:                          Logit   Df Residuals:                      265
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.2791
Time:                        23:56:39   Log-Likelihood:                -136.32
converged:                       True   LL-Null:                       -189.09
Covariance Type:            nonrobust   LLR p-value:                 8.281e-15
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -4.2753      0.742     -5.758      0.000      -5.730      -2.820
AU01          -1.7066     40.270     -0.042      0.966     -80.633      77.220
AU02         -64.3611

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  284
Model:                          Logit   Df Residuals:                      266
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.2691
Time:                        23:56:39   Log-Likelihood:                -138.56
converged:                       True   LL-Null:                       -189.58
Covariance Type:            nonrobust   LLR p-value:                 3.734e-14
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -4.5628      0.756     -6.036      0.000      -6.044      -3.081
AU01          -6.0111     41.170     -0.146      0.884     -86.703      74.681
AU02         -69.5545     44.069     -1.578      0.1

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  283
Model:                          Logit   Df Residuals:                      265
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.2622
Time:                        23:56:40   Log-Likelihood:                -139.52
converged:                       True   LL-Null:                       -189.09
Covariance Type:            nonrobust   LLR p-value:                 1.282e-13
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -4.2776      0.754     -5.676      0.000      -5.755      -2.800
AU01         -10.0101     40.653     -0.246      0.806     -89.689      69.668
AU02         -56.3433     42.231     -1.334      0.1

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  283
Model:                          Logit   Df Residuals:                      265
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.2517
Time:                        23:56:40   Log-Likelihood:                -141.50
converged:                       True   LL-Null:                       -189.09
Covariance Type:            nonrobust   LLR p-value:                 6.905e-13
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -4.2944      0.734     -5.849      0.000      -5.733      -2.855
AU01          -2.1248     40.060     -0.053      0.958     -80.642      76.392
AU02         -60.7373     42.398     -1.433      0.1

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  283
Model:                          Logit   Df Residuals:                      265
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.3044
Time:                        23:56:40   Log-Likelihood:                -131.53
converged:                       True   LL-Null:                       -189.09
Covariance Type:            nonrobust   LLR p-value:                 1.299e-16
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -5.1754      0.817     -6.338      0.000      -6.776      -3.575
AU01           2.5707     41.754      0.062      0.951     -79.265      84.407
AU02         -73.3143     44.927     -1.632      0.1

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  283
Model:                          Logit   Df Residuals:                      265
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.3011
Time:                        23:56:40   Log-Likelihood:                -128.47
converged:                       True   LL-Null:                       -183.81
Covariance Type:            nonrobust   LLR p-value:                 8.972e-16
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -5.1100      0.837     -6.107      0.000      -6.750      -3.470
AU01          -9.6504     43.822     -0.220      0.826     -95.540      76.239
AU02         -81.1219     44.698     -1.815      0.0

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  283
Model:                          Logit   Df Residuals:                      265
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.2647
Time:                        23:56:41   Log-Likelihood:                -139.04
converged:                       True   LL-Null:                       -189.09
Covariance Type:            nonrobust   LLR p-value:                 8.505e-14
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -4.1093      0.743     -5.533      0.000      -5.565      -2.654
AU01          36.0580     45.263      0.797      0.426     -52.656     124.772
AU02         -71.7474     42.851     -1.674      0.0

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# 削除する列を指定
columns_to_drop = ['Confidence', 'label', 'AU07', 'AU11', 'AU20']

# グリッドサーチ用のパラメータ設定
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],   # 正則化強度
    'penalty': ['l2'],              # L2正則化のみを使用（L1やElasticNetを使う場合はsolverを変更）
    'solver': ['lbfgs']             # lbfgsはL2正則化に対応
}

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # ロジスティック回帰モデルのグリッドサーチ
    model = LogisticRegression(max_iter=1000)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=-1)
    grid_search.fit(X_train_combined, y_train_combined)
    
    # 最適なモデルを取得
    best_model = grid_search.best_estimator_
    print(f"最適パラメータ: {grid_search.best_params_}")
    
    # テストデータで予測
    y_pred = best_model.predict(X_test)
    y_pred_prob = best_model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")
    print(f"正解ラベル: {y_test.tolist()}")
    print(f"予測ラベル: {y_pred.tolist()}")
    print(f"クラス1の確率: {y_pred_prob.tolist()}")

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")



===== テストデータ: ファイル 1/31 =====
最適パラメータ: {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID30_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.3129090956188107, 0.323096945949799, 0.3366664462723985, 0.2911688778227644, 0.3431246927934277, 0.3166415783897699, 0.32532358856682825, 0.3769784473111178, 0.3267675183914875, 0.3399672256016164]

===== テストデータ: ファイル 2/31 =====
最適パラメータ: {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID31_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.3276388796392917, 0.28734122242439614, 0.31724226962524316, 0.30259142390270133, 0.27599056823172624, 0.3379734879005737, 0.3051911924593177, 0.44451463694979076, 0.3370241877755699, 0.28075181492137735]

===== テストデータ: ファイル 3/31 =====
最適パラメータ: {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID32_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1,

最適パラメータ: {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID50_smoothed.csv
予測率: 40.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 1, 0, 1, 1, 1, 1, 1, 0]
クラス1の確率: [0.4698613677048706, 0.4990039295188257, 0.6550480050923035, 0.46568553991453643, 0.5823847175769842, 0.635436416470679, 0.6101972497947644, 0.6450167380706455, 0.6594903223315138, 0.48857389975150356]

===== テストデータ: ファイル 22/31 =====
最適パラメータ: {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID54_smoothed.csv
予測率: 55.56%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 1, 1, 0, 1, 1, 1, 0]
クラス1の確率: [0.4054928092157099, 0.3942692079041018, 0.5409214008917143, 0.5276057718711505, 0.4713280120261612, 0.8441814662964595, 0.5809583267175314, 0.6271979808688081, 0.42718480898640715]

===== テストデータ: ファイル 23/31 =====
最適パラメータ: {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID56_smoothed.csv
予測率: 14.29%
正解ラベル: [1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 1]
クラス1の確率: [0.377093385966

## statsmodels（フィルタ後の先頭30フレームの最大値）

In [11]:
# 処理するフォルダのパス
folder_path = './output_moving_filtered_data'

# フォルダ内のすべてのCSVファイルを取得
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
labels = [0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,1,1,1,1,0,1,0,0,0]
i = 0
# 各ファイルに対して処理を適用
result_list = []  # 平均値を保存するリスト

for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    
    # データの読み込み
    df = pd.read_csv(file_path, low_memory=False)
    
    # `question1`から`question10`のみをフィルタリング
    filtered_df = df[df["label"].str.match(r'^question(10|[1-9])$', na=False)]
    
    # Confidenceが0.90以上の行を抽出
    filtered_df = filtered_df[filtered_df["Confidence"] >= 0.90]
    
    # `Confidence`列と`AU`で始まる列を抽出
    columns_to_extract = ["label", "Confidence"] + [col for col in filtered_df.columns if col.startswith("AU")]
    filtered_df = filtered_df[columns_to_extract]
    
    # 順序を保つためのカスタム順序のカテゴリを定義
    filtered_df["label"] = pd.Categorical(
        filtered_df["label"],
        categories=[f"question{i}" for i in range(1, 11)],
        ordered=True
    )
    
    # ラベルごとに最初の30フレームを取得
    first_30_frames_per_label = (
        filtered_df.groupby("label")
        .apply(lambda group: group.head(30))
        .reset_index(drop=True)  # グループ化インデックスをリセット
    )

    # ラベルごとに30フレームの最大値を計算
    max_values_per_label = (
        first_30_frames_per_label.groupby("label").max(numeric_only=True)
    )
    
    # 結果をリストに保存
    result_list.append((file_name, max_values_per_label))

    # Yを追加
    max_values_per_label["Y"] = labels[i]
    i = i + 1
    
# 各ファイルの結果を出力
for file_name, mean_values in result_list:
    print(f"Results for {file_name}:")
    print(mean_values)
    print()

Results for filtered_ID30_smoothed.csv:
            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.998051  0.629318  0.309042  0.560439  0.350857  0.226028   
question2     0.998089  0.709137  0.531324  0.592470  0.349076  0.210350   
question3     0.998311  0.679692  0.416384  0.495837  0.358993  0.189606   
question4     0.998152  0.653097  0.327153  0.503030  0.373655  0.200907   
question5     0.997102  0.648369  0.333211  0.637200  0.354853  0.362731   
question6     0.998499  0.672398  0.391179  0.546530  0.479017  0.273631   
question7     0.997608  0.690041  0.391348  0.550019  0.489479  0.240377   
question8     0.998259  0.596494  0.398190  0.536899  0.332791  0.330788   
question9     0.998850  0.773111  0.433624  0.547648  0.425296  0.241967   
question10    0.998399  0.773570  0.443391  0.571621  0.455789  0.209519   

                AU09      AU10      AU12      A

            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1          NaN       NaN       NaN       NaN       NaN       NaN   
question2     0.999444  0.795853  0.747439  0.156035  0.413963  0.115866   
question3     0.999543  0.814837  0.726615  0.344338  0.467472  0.121091   
question4     0.999673  0.805932  0.638270  0.413653  0.433450  0.112181   
question5     0.999624  0.785121  0.659937  0.403768  0.390078  0.119926   
question6     0.999600  0.809432  0.694735  0.409710  0.441513  0.118071   
question7     0.999458  0.832614  0.775900  0.460733  0.410375  0.152680   
question8     0.999569  0.825975  0.753480  0.311757  0.461633  0.116173   
question9     0.999556  0.780806  0.542794  0.373463  0.365632  0.137501   
question10    0.999638  0.737601  0.572899  0.361941  0.313953  0.120596   

                AU09      AU10      AU12      AU14      AU15      AU17  \
label        

In [16]:
# 削除する列を指定
columns_to_drop = ['Confidence','label']

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # 定数列を追加
    X_train_combined = sm.add_constant(X_train_combined)
    X_test = sm.add_constant(X_test)
    
    # ロジスティック回帰モデルの作成と訓練
    model = sm.Logit(y_train_combined, X_train_combined)
    result = model.fit(disp=False)
    print(result.summary())
    
    # 予測（確率）
    y_pred_prob = result.predict(X_test)
    
    # 確率を0または1に変換
    y_pred = (y_pred_prob >= 0.5).astype(int)
    
    # 精度計算
    correct_predictions = (y_pred == y_test).sum()
    accuracy = (correct_predictions / len(y_test)) * 100  # 予測率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")  # 小数点以下2桁で%表示
    print(f"正解ラベル: {y_test.tolist()}")  # 正解ラベルをリストとして表示
    print(f"予測ラベル: {y_pred.tolist()}")  # 予測ラベルをリストとして表示

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  283
Model:                          Logit   Df Residuals:                      265
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.3405
Time:                        23:56:47   Log-Likelihood:                -124.70
converged:                       True   LL-Null:                       -189.09
Covariance Type:            nonrobust   LLR p-value:                 3.216e-19
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          7.4246      2.704      2.746      0.006       2.125      12.724
AU01          -7.7772      2.880     -2.700      0.007     -13.423      -2.132
AU02           6.3178

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  284
Model:                          Logit   Df Residuals:                      266
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.3447
Time:                        23:56:48   Log-Likelihood:                -124.23
converged:                       True   LL-Null:                       -189.58
Covariance Type:            nonrobust   LLR p-value:                 1.368e-19
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          8.0760      2.716      2.974      0.003       2.753      13.399
AU01          -7.3054      2.852     -2.561      0.010     -12.896      -1.715
AU02           6.5772      2.099      3.133      0.0

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  283
Model:                          Logit   Df Residuals:                      265
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.3406
Time:                        23:56:48   Log-Likelihood:                -124.69
converged:                       True   LL-Null:                       -189.09
Covariance Type:            nonrobust   LLR p-value:                 3.189e-19
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          7.1049      2.726      2.606      0.009       1.761      12.449
AU01          -5.8881      2.899     -2.031      0.042     -11.570      -0.206
AU02           5.7940      2.050      2.826      0.0

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  284
Model:                          Logit   Df Residuals:                      266
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.3850
Time:                        23:56:48   Log-Likelihood:                -113.68
converged:                       True   LL-Null:                       -184.85
Covariance Type:            nonrobust   LLR p-value:                 7.665e-22
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          4.8242      2.881      1.675      0.094      -0.822      10.470
AU01          -6.5645      2.945     -2.229      0.026     -12.337      -0.792
AU02           5.0953      2.131      2.391      0.0

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  283
Model:                          Logit   Df Residuals:                      265
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.3577
Time:                        23:56:48   Log-Likelihood:                -118.06
converged:                       True   LL-Null:                       -183.81
Covariance Type:            nonrobust   LLR p-value:                 9.615e-20
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         11.5039      2.974      3.868      0.000       5.675      17.333
AU01          -7.2020      2.926     -2.462      0.014     -12.937      -1.467
AU02           6.4096      2.149      2.982      0.0

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  283
Model:                          Logit   Df Residuals:                      265
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.3350
Time:                        23:56:49   Log-Likelihood:                -122.24
converged:                       True   LL-Null:                       -183.81
Covariance Type:            nonrobust   LLR p-value:                 3.884e-18
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          6.1400      2.791      2.200      0.028       0.669      11.611
AU01          -6.2070      2.954     -2.101      0.036     -11.997      -0.417
AU02           6.3433      2.131      2.977      0.0

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  283
Model:                          Logit   Df Residuals:                      265
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.3415
Time:                        23:56:49   Log-Likelihood:                -124.51
converged:                       True   LL-Null:                       -189.09
Covariance Type:            nonrobust   LLR p-value:                 2.714e-19
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          8.4531      2.694      3.138      0.002       3.174      13.732
AU01          -7.4107      2.834     -2.615      0.009     -12.966      -1.855
AU02           6.5485      2.090      3.133      0.0

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# 削除する列を指定
columns_to_drop = ['Confidence', 'label', 'AU07', 'AU11', 'AU20']

# グリッドサーチ用のパラメータ設定
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],   # 正則化強度
    'penalty': ['l2'],              # L2正則化のみを使用（L1やElasticNetを使う場合はsolverを変更）
    'solver': ['lbfgs']             # lbfgsはL2正則化に対応
}

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # ロジスティック回帰モデルのグリッドサーチ
    model = LogisticRegression(max_iter=1000)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=-1)
    grid_search.fit(X_train_combined, y_train_combined)
    
    # 最適なモデルを取得
    best_model = grid_search.best_estimator_
    print(f"最適パラメータ: {grid_search.best_params_}")
    
    # テストデータで予測
    y_pred = best_model.predict(X_test)
    y_pred_prob = best_model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")
    print(f"正解ラベル: {y_test.tolist()}")
    print(f"予測ラベル: {y_pred.tolist()}")
    print(f"クラス1の確率: {y_pred_prob.tolist()}")

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")



===== テストデータ: ファイル 1/31 =====
最適パラメータ: {'C': 10, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID30_smoothed.csv
予測率: 60.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 1, 0, 0, 1, 0, 1, 1, 0, 0]
クラス1の確率: [0.3570850602416358, 0.5179376075173054, 0.35594125050811554, 0.2208640142455177, 0.6285465399509286, 0.4411158335307798, 0.5080511680179055, 0.5755312213595126, 0.45190758125767855, 0.46753896590561295]

===== テストデータ: ファイル 2/31 =====
最適パラメータ: {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID31_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.32196208022122114, 0.20069666744415268, 0.14137596931683186, 0.26721988660655793, 0.29114615672002875, 0.3228785128376643, 0.23535720453283626, 0.26965162659946024, 0.21611035655022642, 0.21719067811655476]

===== テストデータ: ファイル 3/31 =====
最適パラメータ: {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID32_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1,

最適パラメータ: {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID54_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.08158863495376122, 0.14369540770532505, 0.2482860728777192, 0.30456179377903103, 0.2713627794101629, 0.3660679692465082, 0.31848581771094725, 0.32065488544361664, 0.2617455675898152]

===== テストデータ: ファイル 23/31 =====
最適パラメータ: {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID56_smoothed.csv
予測率: 100.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1]
予測ラベル: [1, 1, 1, 1, 1, 1, 1]
クラス1の確率: [0.6072925556815014, 0.5720105389861456, 0.5229099140898094, 0.6897359251129692, 0.5970308065015666, 0.649167056844221, 0.5200458817857468]

===== テストデータ: ファイル 24/31 =====
最適パラメータ: {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID57_smoothed.csv
予測率: 10.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.5200836722785139, 0.49531657854219413, 0.2996874837076901, 0.370995778115559

## statsmodels（フィルタ後の先頭10フレームの最大値）

In [13]:
# 処理するフォルダのパス
folder_path = './output_moving_filtered_data'

# フォルダ内のすべてのCSVファイルを取得
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
labels = [0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,1,1,1,1,0,1,0,0,0]
i = 0
# 各ファイルに対して処理を適用
result_list = []  # 平均値を保存するリスト

for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    
    # データの読み込み
    df = pd.read_csv(file_path, low_memory=False)
    
    # `question1`から`question10`のみをフィルタリング
    filtered_df = df[df["label"].str.match(r'^question(10|[1-9])$', na=False)]
    
    # Confidenceが0.90以上の行を抽出
    filtered_df = filtered_df[filtered_df["Confidence"] >= 0.90]
    
    # `Confidence`列と`AU`で始まる列を抽出
    columns_to_extract = ["label", "Confidence"] + [col for col in filtered_df.columns if col.startswith("AU")]
    filtered_df = filtered_df[columns_to_extract]
    
    # 順序を保つためのカスタム順序のカテゴリを定義
    filtered_df["label"] = pd.Categorical(
        filtered_df["label"],
        categories=[f"question{i}" for i in range(1, 11)],
        ordered=True
    )
    
    # ラベルごとに最初の10フレームを取得
    first_10_frames_per_label = (
        filtered_df.groupby("label")
        .apply(lambda group: group.head(10))
        .reset_index(drop=True)  # グループ化インデックスをリセット
    )

    # ラベルごとに10フレームの最大値を計算
    max_values_per_label = (
        first_10_frames_per_label.groupby("label").max(numeric_only=True)
    )
    
    # 結果をリストに保存
    result_list.append((file_name, max_values_per_label))

    # Yを追加
    max_values_per_label["Y"] = labels[i]
    i = i + 1
    
# 各ファイルの結果を出力
for file_name, mean_values in result_list:
    print(f"Results for {file_name}:")
    print(mean_values)
    print()

Results for filtered_ID30_smoothed.csv:
            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.996113  0.574940  0.309042  0.455083  0.350857  0.226028   
question2     0.996619  0.709137  0.531324  0.592470  0.301983  0.210350   
question3     0.998311  0.666825  0.361777  0.495837  0.347845  0.174839   
question4     0.997968  0.619893  0.321957  0.503030  0.348712  0.200907   
question5     0.997069  0.606037  0.327818  0.637200  0.342222  0.359800   
question6     0.998499  0.620801  0.391179  0.546530  0.458388  0.273631   
question7     0.996357  0.690041  0.391348  0.550019  0.436465  0.240377   
question8     0.998259  0.577708  0.339130  0.536899  0.330281  0.277568   
question9     0.998850  0.727150  0.433624  0.488248  0.425296  0.239437   
question10    0.998399  0.731874  0.439494  0.484566  0.455789  0.202454   

                AU09      AU10      AU12      A

            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.996899  0.551931  0.526094  0.328334  0.308077  0.261550   
question2     0.990769  0.749801  0.576685  0.391308  0.432102  0.186012   
question3     0.996099  0.570140  0.475438  0.548116  0.425159  0.135526   
question4     0.992315  0.728729  0.570797  0.546687  0.381186  0.107742   
question5     0.976713  0.671322  0.445506  0.494419  0.307890  0.154849   
question6     0.987861  0.606000  0.423162  0.498880  0.469501  0.139714   
question7     0.994584  0.514369  0.354218  0.564874  0.377324  0.245548   
question8     0.996840  0.550625  0.458617  0.505911  0.378698  0.233880   
question9     0.994206  0.565095  0.439749  0.530077  0.345585  0.561359   
question10    0.982216  0.665805  0.474969  0.358670  0.389510  0.254809   

                AU09      AU10      AU12      AU14      AU15      AU17  \
label        

In [18]:
# 削除する列を指定
columns_to_drop = ['Confidence','label']

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # 定数列を追加
    X_train_combined = sm.add_constant(X_train_combined)
    X_test = sm.add_constant(X_test)
    
    # ロジスティック回帰モデルの作成と訓練
    model = sm.Logit(y_train_combined, X_train_combined)
    result = model.fit(disp=False)
    print(result.summary())
    
    # 予測（確率）
    y_pred_prob = result.predict(X_test)
    
    # 確率を0または1に変換
    y_pred = (y_pred_prob >= 0.5).astype(int)
    
    # 精度計算
    correct_predictions = (y_pred == y_test).sum()
    accuracy = (correct_predictions / len(y_test)) * 100  # 予測率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")  # 小数点以下2桁で%表示
    print(f"正解ラベル: {y_test.tolist()}")  # 正解ラベルをリストとして表示
    print(f"予測ラベル: {y_pred.tolist()}")  # 予測ラベルをリストとして表示

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  283
Model:                          Logit   Df Residuals:                      265
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.2746
Time:                        23:56:56   Log-Likelihood:                -137.17
converged:                       True   LL-Null:                       -189.09
Covariance Type:            nonrobust   LLR p-value:                 1.725e-14
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          5.1130      2.543      2.010      0.044       0.128      10.098
AU01          -5.9495      2.389     -2.490      0.013     -10.632      -1.267
AU02           4.7919

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  284
Model:                          Logit   Df Residuals:                      266
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.2849
Time:                        23:56:56   Log-Likelihood:                -135.56
converged:                       True   LL-Null:                       -189.58
Covariance Type:            nonrobust   LLR p-value:                 2.819e-15
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          6.3490      2.596      2.446      0.014       1.261      11.437
AU01          -5.8325      2.410     -2.421      0.015     -10.555      -1.110
AU02           5.4223      1.870      2.900      0.0

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  283
Model:                          Logit   Df Residuals:                      265
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.2874
Time:                        23:56:56   Log-Likelihood:                -134.74
converged:                       True   LL-Null:                       -189.09
Covariance Type:            nonrobust   LLR p-value:                 2.118e-15
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          5.8207      2.584      2.253      0.024       0.757      10.885
AU01          -5.1037      2.472     -2.064      0.039      -9.949      -0.258
AU02           4.9257      1.833      2.687      0.0

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  284
Model:                          Logit   Df Residuals:                      266
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.3246
Time:                        23:56:56   Log-Likelihood:                -124.84
converged:                       True   LL-Null:                       -184.85
Covariance Type:            nonrobust   LLR p-value:                 1.540e-17
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.6253      2.756      0.590      0.555      -3.776       7.027
AU01          -4.6624      2.455     -1.899      0.058      -9.473       0.149
AU02           3.8156      1.857      2.054      0.0

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  283
Model:                          Logit   Df Residuals:                      265
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.2780
Time:                        23:56:56   Log-Likelihood:                -132.70
converged:                       True   LL-Null:                       -183.81
Covariance Type:            nonrobust   LLR p-value:                 3.452e-14
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          8.8571      2.793      3.172      0.002       3.384      14.331
AU01          -5.2435      2.390     -2.194      0.028      -9.927      -0.560
AU02           4.1798      1.826      2.289      0.0

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  283
Model:                          Logit   Df Residuals:                      265
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.2753
Time:                        23:56:57   Log-Likelihood:                -133.22
converged:                       True   LL-Null:                       -183.81
Covariance Type:            nonrobust   LLR p-value:                 5.359e-14
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          3.6536      2.615      1.397      0.162      -1.472       8.779
AU01          -3.7476      2.448     -1.531      0.126      -8.545       1.050
AU02           4.5271      1.841      2.460      0.0

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  283
Model:                          Logit   Df Residuals:                      265
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.2735
Time:                        23:56:57   Log-Likelihood:                -137.37
converged:                       True   LL-Null:                       -189.09
Covariance Type:            nonrobust   LLR p-value:                 2.038e-14
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          6.3454      2.553      2.486      0.013       1.342      11.349
AU01          -5.5375      2.362     -2.344      0.019     -10.168      -0.907
AU02           4.7200      1.801      2.620      0.0

In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# 削除する列を指定
columns_to_drop = ['Confidence', 'label', 'AU07', 'AU11', 'AU20']

# グリッドサーチ用のパラメータ設定
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],   # 正則化強度
    'penalty': ['l2'],              # L2正則化のみを使用（L1やElasticNetを使う場合はsolverを変更）
    'solver': ['lbfgs']             # lbfgsはL2正則化に対応
}

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # ロジスティック回帰モデルのグリッドサーチ
    model = LogisticRegression(max_iter=1000)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=-1)
    grid_search.fit(X_train_combined, y_train_combined)
    
    # 最適なモデルを取得
    best_model = grid_search.best_estimator_
    print(f"最適パラメータ: {grid_search.best_params_}")
    
    # テストデータで予測
    y_pred = best_model.predict(X_test)
    y_pred_prob = best_model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")
    print(f"正解ラベル: {y_test.tolist()}")
    print(f"予測ラベル: {y_pred.tolist()}")
    print(f"クラス1の確率: {y_pred_prob.tolist()}")

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")



===== テストデータ: ファイル 1/31 =====
最適パラメータ: {'C': 10, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID30_smoothed.csv
予測率: 80.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 1, 0, 0, 1, 0, 0, 0, 0, 0]
クラス1の確率: [0.40408288937184833, 0.504997617005214, 0.30684593324814535, 0.22945714978183102, 0.6000270782249477, 0.4527236627233177, 0.4602785250523788, 0.49559629114931775, 0.47376000414917324, 0.4903487590795495]

===== テストデータ: ファイル 2/31 =====
最適パラメータ: {'C': 10, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID31_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.15120392112395595, 0.07612002465453081, 0.048531906114240096, 0.14971582337186998, 0.11264428922134934, 0.12009948763408461, 0.09724693469436232, 0.1590400471502398, 0.09819350856809911, 0.087667949303545]

===== テストデータ: ファイル 3/31 =====
最適パラメータ: {'C': 10, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID32_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 

最適パラメータ: {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID54_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.09850881842709121, 0.1261022090239585, 0.2171128397926334, 0.28642747578179717, 0.2999814246314648, 0.34041370694762624, 0.36207631774516846, 0.2860016404503246, 0.34202076907060514]

===== テストデータ: ファイル 23/31 =====
最適パラメータ: {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID56_smoothed.csv
予測率: 85.71%
正解ラベル: [1, 1, 1, 1, 1, 1, 1]
予測ラベル: [1, 1, 1, 1, 1, 1, 0]
クラス1の確率: [0.5420212719600848, 0.6269740146949703, 0.6096148703643213, 0.6473818685186115, 0.5386659234257826, 0.5852100065639676, 0.47738379186383384]

===== テストデータ: ファイル 24/31 =====
最適パラメータ: {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID57_smoothed.csv
予測率: 10.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
クラス1の確率: [0.4461580896261092, 0.40252409976322234, 0.22028475526708588, 0.3904147824950

## statsmodels（フィルタ後の先頭30フレームの最大値と最小値の差分）

In [15]:
# 処理するフォルダのパス
folder_path = './output_moving_filtered_data'

# フォルダ内のすべてのCSVファイルを取得
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
labels = [0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,1,1,1,1,0,1,0,0,0]
i = 0
# 各ファイルに対して処理を適用
result_list = []  # 平均値を保存するリスト

for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    
    # データの読み込み
    df = pd.read_csv(file_path, low_memory=False)
    
    # `question1`から`question10`のみをフィルタリング
    filtered_df = df[df["label"].str.match(r'^question(10|[1-9])$', na=False)]
    
    # Confidenceが0.90以上の行を抽出
    filtered_df = filtered_df[filtered_df["Confidence"] >= 0.90]
    
    # `Confidence`列と`AU`で始まる列を抽出
    columns_to_extract = ["label", "Confidence"] + [col for col in filtered_df.columns if col.startswith("AU")]
    filtered_df = filtered_df[columns_to_extract]
    
    # 順序を保つためのカスタム順序のカテゴリを定義
    filtered_df["label"] = pd.Categorical(
        filtered_df["label"],
        categories=[f"question{i}" for i in range(1, 11)],
        ordered=True
    )
    
    # ラベルごとに最初の30フレームを取得
    first_30_frames_per_label = (
        filtered_df.groupby("label")
        .apply(lambda group: group.head(30))
        .reset_index(drop=True)  # グループ化インデックスをリセット
    )

    # ラベルごとに最大値を計算
    max_values_per_label = (
        first_30_frames_per_label.groupby("label").max(numeric_only=True)
    )

    # ラベルごとに最小値を計算
    min_values_per_label = (
        first_30_frames_per_label.groupby("label").min(numeric_only=True)
    )

    # 最大値と最小値の差分を計算
    range_values_per_label = max_values_per_label - min_values_per_label

    
    # 結果をリストに保存
    result_list.append((file_name, range_values_per_label))

    # Yを追加
    range_values_per_label["Y"] = labels[i]
    i = i + 1
    
# 各ファイルの結果を出力
for file_name, mean_values in result_list:
    print(f"Results for {file_name}:")
    print(mean_values)
    print()

Results for filtered_ID30_smoothed.csv:
            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.005760  0.091032  0.089920  0.191338  0.080192  0.074836   
question2     0.005446  0.148309  0.230588  0.127627  0.091902  0.060158   
question3     0.007495  0.063130  0.116779  0.076798  0.059728  0.049990   
question4     0.006056  0.085221  0.054412  0.130832  0.095369  0.034621   
question5     0.005688  0.059413  0.127185  0.122237  0.056385  0.113033   
question6     0.008136  0.088952  0.054877  0.111280  0.078245  0.071187   
question7     0.009249  0.099774  0.071603  0.178759  0.139358  0.056048   
question8     0.005283  0.043323  0.144713  0.144408  0.031742  0.069094   
question9     0.004680  0.101396  0.100224  0.097717  0.107856  0.035054   
question10    0.005946  0.062949  0.066295  0.180012  0.138571  0.024668   

                AU09      AU10      AU12      A

            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.006615  0.052072  0.040254  0.065752  0.120005  0.051395   
question2     0.010975  0.024418  0.062088  0.145836  0.114290  0.023101   
question3     0.000770  0.038822  0.094662  0.144699  0.027944  0.178940   
question4     0.000902  0.056666  0.072145  0.244472  0.060213  0.206722   
question5     0.000639  0.049071  0.091725  0.048452  0.012811  0.086924   
question6     0.000971  0.058235  0.026088  0.151193  0.041501  0.126670   
question7     0.003688  0.094019  0.114582  0.176200  0.022755  0.043200   
question8     0.000736  0.030343  0.066578  0.201996  0.011826  0.146655   
question9     0.000345  0.019049  0.045554  0.186206  0.058488  0.053434   
question10    0.000219  0.035130  0.069906  0.234624  0.013784  0.037261   

                AU09      AU10      AU12      AU14      AU15      AU17  \
label        

            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.063389  0.226203  0.207170  0.149517  0.196599  0.443126   
question2     0.078009  0.171396  0.182776  0.323207  0.149670  0.176866   
question3          NaN       NaN       NaN       NaN       NaN       NaN   
question4     0.039868  0.130669  0.291182  0.274616  0.150800  0.140695   
question5     0.092295  0.207035  0.166137  0.269783  0.048590  0.336453   
question6     0.050413  0.114305  0.092336  0.313250  0.092341  0.192002   
question7     0.031394  0.295390  0.292734  0.444508  0.136674  0.103904   
question8     0.027517  0.308155  0.421877  0.374284  0.131084  0.595248   
question9     0.024593  0.060609  0.188337  0.161420  0.066388  0.177290   
question10    0.081387  0.277482  0.199909  0.205849  0.097019  0.095391   

                AU09      AU10      AU12      AU14      AU15      AU17  \
label        

In [20]:
# 削除する列を指定
columns_to_drop = ['Confidence','label']

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # 定数列を追加
    X_train_combined = sm.add_constant(X_train_combined)
    X_test = sm.add_constant(X_test)
    
    # ロジスティック回帰モデルの作成と訓練
    model = sm.Logit(y_train_combined, X_train_combined)
    result = model.fit(disp=False)
    print(result.summary())
    
    # 予測（確率）
    y_pred_prob = result.predict(X_test)
    
    # 確率を0または1に変換
    y_pred = (y_pred_prob >= 0.5).astype(int)
    
    # 精度計算
    correct_predictions = (y_pred == y_test).sum()
    accuracy = (correct_predictions / len(y_test)) * 100  # 予測率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")  # 小数点以下2桁で%表示
    print(f"正解ラベル: {y_test.tolist()}")  # 正解ラベルをリストとして表示
    print(f"予測ラベル: {y_pred.tolist()}")  # 予測ラベルをリストとして表示

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  283
Model:                          Logit   Df Residuals:                      265
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.1669
Time:                        23:57:04   Log-Likelihood:                -157.52
converged:                       True   LL-Null:                       -189.09
Covariance Type:            nonrobust   LLR p-value:                 3.163e-07
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.6913      0.539     -3.137      0.002      -2.748      -0.635
AU01          -1.2623      2.993     -0.422      0.673      -7.128       4.604
AU02          -2.7609

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  285
Model:                          Logit   Df Residuals:                      267
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.1474
Time:                        23:57:04   Log-Likelihood:                -158.48
converged:                       True   LL-Null:                       -185.88
Covariance Type:            nonrobust   LLR p-value:                 7.406e-06
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.9549      0.539     -3.629      0.000      -3.011      -0.899
AU01          -1.5128      3.007     -0.503      0.615      -7.407       4.382
AU02          -1.8388      2.772     -0.663      0.5

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  283
Model:                          Logit   Df Residuals:                      265
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.1517
Time:                        23:57:05   Log-Likelihood:                -160.41
converged:                       True   LL-Null:                       -189.09
Covariance Type:            nonrobust   LLR p-value:                 2.840e-06
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.7919      0.538     -3.329      0.001      -2.847      -0.737
AU01          -0.2479      3.103     -0.080      0.936      -6.329       5.834
AU02          -2.5864      2.849     -0.908      0.3

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  284
Model:                          Logit   Df Residuals:                      266
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.1958
Time:                        23:57:05   Log-Likelihood:                -148.65
converged:                       True   LL-Null:                       -184.85
Covariance Type:            nonrobust   LLR p-value:                 8.346e-09
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.3367      0.578     -4.040      0.000      -3.470      -1.203
AU01          -0.2985      3.080     -0.097      0.923      -6.335       5.738
AU02          -1.4678      2.898     -0.506      0.6

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  286
Model:                          Logit   Df Residuals:                      268
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.1657
Time:                        23:57:05   Log-Likelihood:                -158.98
converged:                       True   LL-Null:                       -190.56
Covariance Type:            nonrobust   LLR p-value:                 3.134e-07
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.0183      0.542     -3.726      0.000      -3.080      -0.957
AU01          -1.1126      3.012     -0.369      0.712      -7.017       4.791
AU02          -2.4729      2.825     -0.875      0.3

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  284
Model:                          Logit   Df Residuals:                      266
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.1736
Time:                        23:57:05   Log-Likelihood:                -152.76
converged:                       True   LL-Null:                       -184.85
Covariance Type:            nonrobust   LLR p-value:                 2.124e-07
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.3173      0.569     -4.075      0.000      -3.432      -1.203
AU01          -0.8984      3.066     -0.293      0.769      -6.908       5.111
AU02          -2.4233      2.820     -0.859      0.3

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  284
Model:                          Logit   Df Residuals:                      266
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.1728
Time:                        23:57:06   Log-Likelihood:                -156.82
converged:                       True   LL-Null:                       -189.58
Covariance Type:            nonrobust   LLR p-value:                 1.255e-07
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.9052      0.548     -3.475      0.001      -2.980      -0.831
AU01          -1.6489      3.071     -0.537      0.591      -7.668       4.370
AU02          -1.2997      2.858     -0.455      0.6

In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# 削除する列を指定
columns_to_drop = ['Confidence', 'label', 'AU07', 'AU11', 'AU20']

# グリッドサーチ用のパラメータ設定
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],   # 正則化強度
    'penalty': ['l2'],              # L2正則化のみを使用（L1やElasticNetを使う場合はsolverを変更）
    'solver': ['lbfgs']             # lbfgsはL2正則化に対応
}

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # ロジスティック回帰モデルのグリッドサーチ
    model = LogisticRegression(max_iter=1000)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=-1)
    grid_search.fit(X_train_combined, y_train_combined)
    
    # 最適なモデルを取得
    best_model = grid_search.best_estimator_
    print(f"最適パラメータ: {grid_search.best_params_}")
    
    # テストデータで予測
    y_pred = best_model.predict(X_test)
    y_pred_prob = best_model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")
    print(f"正解ラベル: {y_test.tolist()}")
    print(f"予測ラベル: {y_pred.tolist()}")
    print(f"クラス1の確率: {y_pred_prob.tolist()}")

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")



===== テストデータ: ファイル 1/31 =====
最適パラメータ: {'C': 10, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID30_smoothed.csv
予測率: 90.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.5243967909611496, 0.3592915520606913, 0.3896433028553761, 0.37447685682708326, 0.2990371022247245, 0.2965280979819352, 0.4498125950969555, 0.3972771699987119, 0.3788306561155249, 0.48076927374842854]

===== テストデータ: ファイル 2/31 =====
最適パラメータ: {'C': 10, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID31_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.33933514492008177, 0.3132994534052821, 0.36514797859885784, 0.2781851944950611, 0.28234961153642, 0.42212138242181907, 0.3239242436475915, 0.42234841269262374, 0.35926127426967525, 0.4592683194843884]

===== テストデータ: ファイル 3/31 =====
最適パラメータ: {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID32_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 

最適パラメータ: {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID50_smoothed.csv
予測率: 80.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 1, 1, 0]
クラス1の確率: [0.40440844584594315, 0.36652187533574015, 0.4653844001606788, 0.49343574445538624, 0.4176323578258583, 0.4283118362940496, 0.4594433181028787, 0.5192258101792718, 0.5536851840184382, 0.4501918502299499]

===== テストデータ: ファイル 22/31 =====
最適パラメータ: {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID54_smoothed.csv
予測率: 11.11%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 1, 0, 0, 0]
クラス1の確率: [0.3703609760501148, 0.32068667851288174, 0.4344598372736681, 0.43277287104796947, 0.343385251001773, 0.5289451226282754, 0.3470301056709518, 0.44205892284421305, 0.31976783038475176]

===== テストデータ: ファイル 23/31 =====
最適パラメータ: {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID56_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.358629767406

## statsmodels（フィルタ後の先頭10フレームの最大値と最小値の差分）

In [17]:
# 処理するフォルダのパス
folder_path = './output_moving_filtered_data'

# フォルダ内のすべてのCSVファイルを取得
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
labels = [0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,1,1,1,1,0,1,0,0,0]
i = 0
# 各ファイルに対して処理を適用
result_list = []  # 平均値を保存するリスト

for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    
    # データの読み込み
    df = pd.read_csv(file_path, low_memory=False)
    
    # `question1`から`question10`のみをフィルタリング
    filtered_df = df[df["label"].str.match(r'^question(10|[1-9])$', na=False)]
    
    # Confidenceが0.90以上の行を抽出
    filtered_df = filtered_df[filtered_df["Confidence"] >= 0.90]
    
    # `Confidence`列と`AU`で始まる列を抽出
    columns_to_extract = ["label", "Confidence"] + [col for col in filtered_df.columns if col.startswith("AU")]
    filtered_df = filtered_df[columns_to_extract]
    
    # 順序を保つためのカスタム順序のカテゴリを定義
    filtered_df["label"] = pd.Categorical(
        filtered_df["label"],
        categories=[f"question{i}" for i in range(1, 11)],
        ordered=True
    )
    
    # ラベルごとに最初の10フレームを取得
    first_10_frames_per_label = (
        filtered_df.groupby("label")
        .apply(lambda group: group.head(10))
        .reset_index(drop=True)  # グループ化インデックスをリセット
    )

    # ラベルごとに最大値を計算
    max_values_per_label = (
        first_10_frames_per_label.groupby("label").max(numeric_only=True)
    )

    # ラベルごとに最小値を計算
    min_values_per_label = (
        first_10_frames_per_label.groupby("label").min(numeric_only=True)
    )

    # 最大値と最小値の差分を計算
    range_values_per_label = max_values_per_label - min_values_per_label

    
    # 結果をリストに保存
    result_list.append((file_name, range_values_per_label))

    # Yを追加
    range_values_per_label["Y"] = labels[i]
    i = i + 1
    
# 各ファイルの結果を出力
for file_name, mean_values in result_list:
    print(f"Results for {file_name}:")
    print(mean_values)
    print()

Results for filtered_ID30_smoothed.csv:
            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.003822  0.036655  0.030980  0.085982  0.017956  0.038607   
question2     0.003976  0.065868  0.105564  0.091642  0.044809  0.028340   
question3     0.002594  0.026349  0.062172  0.042632  0.047751  0.019277   
question4     0.002409  0.052017  0.049216  0.049436  0.070426  0.030970   
question5     0.005654  0.017081  0.084297  0.122237  0.016569  0.076541   
question6     0.005794  0.008200  0.029056  0.111280  0.038556  0.048539   
question7     0.003267  0.039577  0.055206  0.088458  0.086344  0.024362   
question8     0.002394  0.024537  0.059238  0.144408  0.022217  0.015875   
question9     0.002268  0.055435  0.065355  0.022176  0.094148  0.021973   
question10    0.003167  0.017304  0.039235  0.068160  0.050081  0.017603   

                AU09      AU10      AU12      A

            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.004904  0.033201  0.023853  0.055436  0.004075  0.002882   
question2     0.004301  0.040898  0.010480  0.081685  0.012395  0.007918   
question3     0.001134  0.032897  0.066120  0.058399  0.027184  0.001082   
question4     0.001321  0.050022  0.027345  0.075014  0.010207  0.001159   
question5     0.002628  0.140186  0.152797  0.060576  0.033427  0.006392   
question6     0.001798  0.101126  0.117896  0.131476  0.033804  0.008669   
question7     0.000794  0.033376  0.022130  0.050782  0.022197  0.001259   
question8     0.001446  0.085454  0.080749  0.068346  0.012144  0.005567   
question9     0.001378  0.041783  0.061271  0.033467  0.006865  0.010205   
question10    0.000163  0.026169  0.014686  0.047596  0.012896  0.015084   

                AU09      AU10      AU12      AU14      AU15      AU17  \
label        

In [22]:
# 削除する列を指定
columns_to_drop = ['Confidence','label']

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # 定数列を追加
    X_train_combined = sm.add_constant(X_train_combined)
    X_test = sm.add_constant(X_test)
    
    # ロジスティック回帰モデルの作成と訓練
    model = sm.Logit(y_train_combined, X_train_combined)
    result = model.fit(disp=False)
    print(result.summary())
    
    # 予測（確率）
    y_pred_prob = result.predict(X_test)
    
    # 確率を0または1に変換
    y_pred = (y_pred_prob >= 0.5).astype(int)
    
    # 精度計算
    correct_predictions = (y_pred == y_test).sum()
    accuracy = (correct_predictions / len(y_test)) * 100  # 予測率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")  # 小数点以下2桁で%表示
    print(f"正解ラベル: {y_test.tolist()}")  # 正解ラベルをリストとして表示
    print(f"予測ラベル: {y_pred.tolist()}")  # 予測ラベルをリストとして表示

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  283
Model:                          Logit   Df Residuals:                      265
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.1230
Time:                        23:57:13   Log-Likelihood:                -165.83
converged:                       True   LL-Null:                       -189.09
Covariance Type:            nonrobust   LLR p-value:                 0.0001446
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.1022      0.483     -4.348      0.000      -3.050      -1.155
AU01          -0.4755      4.237     -0.112      0.911      -8.781       7.830
AU02          -5.1979

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  285
Model:                          Logit   Df Residuals:                      267
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.1139
Time:                        23:57:13   Log-Likelihood:                -164.70
converged:                       True   LL-Null:                       -185.88
Covariance Type:            nonrobust   LLR p-value:                 0.0005970
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.2660      0.489     -4.630      0.000      -3.225      -1.307
AU01          -0.3229      4.285     -0.075      0.940      -8.722       8.076
AU02          -4.5047      4.185     -1.076      0.2

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  284
Model:                          Logit   Df Residuals:                      266
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.1205
Time:                        23:57:13   Log-Likelihood:                -166.73
converged:                       True   LL-Null:                       -189.58
Covariance Type:            nonrobust   LLR p-value:                 0.0001916
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.1308      0.487     -4.373      0.000      -3.086      -1.176
AU01          -0.1041      4.249     -0.025      0.980      -8.431       8.223
AU02          -4.6866      4.224     -1.109      0.2

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  283
Model:                          Logit   Df Residuals:                      265
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.1277
Time:                        23:57:13   Log-Likelihood:                -160.33
converged:                       True   LL-Null:                       -183.81
Covariance Type:            nonrobust   LLR p-value:                 0.0001236
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.4744      0.506     -4.891      0.000      -3.466      -1.483
AU01           2.2108      4.237      0.522      0.602      -6.094      10.516
AU02          -4.3597      4.231     -1.030      0.3

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  283
Model:                          Logit   Df Residuals:                      265
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.1158
Time:                        23:57:14   Log-Likelihood:                -162.52
converged:                       True   LL-Null:                       -183.81
Covariance Type:            nonrobust   LLR p-value:                 0.0005531
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.0836      0.493     -4.229      0.000      -3.049      -1.118
AU01           0.9817      4.259      0.230      0.818      -7.367       9.330
AU02          -4.5575      4.187     -1.088      0.2

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  283
Model:                          Logit   Df Residuals:                      265
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.1112
Time:                        23:57:14   Log-Likelihood:                -163.36
converged:                       True   LL-Null:                       -183.81
Covariance Type:            nonrobust   LLR p-value:                 0.0009684
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.0133      0.498     -4.045      0.000      -2.989      -1.038
AU01          -1.5514      4.571     -0.339      0.734     -10.511       7.408
AU02          -5.5781      4.413     -1.264      0.2

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  283
Model:                          Logit   Df Residuals:                      265
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.1126
Time:                        23:57:14   Log-Likelihood:                -167.80
converged:                       True   LL-Null:                       -189.09
Covariance Type:            nonrobust   LLR p-value:                 0.0005534
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.0668      0.482     -4.284      0.000      -3.012      -1.121
AU01           0.6489      4.174      0.155      0.876      -7.532       8.830
AU02          -4.5232      4.144     -1.091      0.2

In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# 削除する列を指定
columns_to_drop = ['Confidence', 'label', 'AU07', 'AU11', 'AU20']

# グリッドサーチ用のパラメータ設定
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],   # 正則化強度
    'penalty': ['l2'],              # L2正則化のみを使用（L1やElasticNetを使う場合はsolverを変更）
    'solver': ['lbfgs']             # lbfgsはL2正則化に対応
}

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # ロジスティック回帰モデルのグリッドサーチ
    model = LogisticRegression(max_iter=1000)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=-1)
    grid_search.fit(X_train_combined, y_train_combined)
    
    # 最適なモデルを取得
    best_model = grid_search.best_estimator_
    print(f"最適パラメータ: {grid_search.best_params_}")
    
    # テストデータで予測
    y_pred = best_model.predict(X_test)
    y_pred_prob = best_model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")
    print(f"正解ラベル: {y_test.tolist()}")
    print(f"予測ラベル: {y_pred.tolist()}")
    print(f"クラス1の確率: {y_pred_prob.tolist()}")

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")



===== テストデータ: ファイル 1/31 =====
最適パラメータ: {'C': 10, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID30_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.460466594724587, 0.45825444361065554, 0.3083726877443414, 0.28438191888501685, 0.4031179459947058, 0.41258338476086287, 0.3668167822505013, 0.42484776004103986, 0.37475598977337593, 0.3509234129964688]

===== テストデータ: ファイル 2/31 =====
最適パラメータ: {'C': 10, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID31_smoothed.csv
予測率: 90.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
クラス1の確率: [0.290407927447111, 0.3100693078450338, 0.4788117065984822, 0.3587689574333876, 0.30203819080698574, 0.2666951326973579, 0.3551023214476346, 0.5560576939888393, 0.3704348475777776, 0.32950350458771416]

===== テストデータ: ファイル 3/31 =====
最適パラメータ: {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID32_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1,

最適パラメータ: {'C': 10, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID50_smoothed.csv
予測率: 40.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 1, 1, 0, 1, 1, 1, 1, 0]
クラス1の確率: [0.3553692205658363, 0.461324592155965, 0.5579992334187616, 0.5424112546390304, 0.47815392921867655, 0.5170864292344232, 0.628059756293396, 0.6546106248073141, 0.5605723612143018, 0.49266739129608444]

===== テストデータ: ファイル 22/31 =====
最適パラメータ: {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID54_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.41470560311912263, 0.337598156939167, 0.43858547238402934, 0.36370413105198723, 0.3460195415480704, 0.4216317134567911, 0.3736778558892396, 0.39064737257846366, 0.3516092237067143]

===== テストデータ: ファイル 23/31 =====
最適パラメータ: {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID56_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.38142077766874516,